# Generate Data for Chartbook

Brian Dew

@bd_econ

In [9]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [2]:
#nipa_series_codes(retrieve_table('T11500'))

### GDPNow

In [10]:
url = ('https://api.stlouisfed.org/fred/series?series_id=GDPNOW&'+
       f'api_key={fred_key}&file_type=json')
r = requests.get(url).json()
update = r['seriess'][0]['last_updated']
url = ('https://api.stlouisfed.org/fred/series/observations?series_id=GDPNOW&'+
       f'api_key={fred_key}&file_type=json')
r = requests.get(url).json()
lv = pd.DataFrame(r['observations']).set_index('date')['value'].sort_index()[-1]
lvt = f'{float(lv):.1f}'
nowdt = pd.DataFrame(r['observations']).set_index('date')['value'].sort_index().index[-1]
ltdt = dtxt(nowdt)['qtr1']

# Store GDP now value
gdpnow = pd.Series({'date': nowdt, 'value': lv}).to_frame().T.set_index('date')
gdpnow.to_csv(data_dir / 'gdpnow.csv', index_label='date')

s = ['A191RL']
gdp = nipa_df(retrieve_table('T10502')['Data'], s).sort_index()
gdpdt = gdp.index[-1]

In [11]:
# Only add mark to plot if nowcast is available
mark = ' (see \\tikz \draw[black, fill=red] (2.5pt,2.5pt) circle (2.5pt);)'
if dtxt(gdpdt)['datetime'] < nowdt:
    linedt = dtxt(pd.to_datetime(nowdt) - pd.DateOffset(days=46))['datetime']
    node = (f'\\node[label={{[align=left]90:{{\scriptsize \\textit{{{lvt}}}}}}}, '+
            'circle, draw=black, fill=red, inner sep=1.8pt] at '+
            f'(axis cs:{nowdt}, {lv}) {{}};'+
            f'\draw [dashed] (axis cs:{{{linedt}}},\pgfkeysvalueof{{/pgfplots/ymin}})'+
            f' -- (axis cs:{{{linedt}}}, \pgfkeysvalueof{{/pgfplots/ymax}});')
    write_txt(text_dir / 'gdpnow_node.txt', node)

    node = ('\\node[circle, draw=black, fill=red, inner sep=1.0pt] '+
            f'at (axis cs:{nowdt}, {lv}) {{}};')
    write_txt(text_dir / 'gdpnow_node2.txt', node)
    # Nowcast text
    write_txt(text_dir / 'nowcast_text.txt', 'nowcast')
    latest_txt = 'latest '

else:
    node = ''
    write_txt(text_dir / 'gdpnow_node.txt', node)
    write_txt(text_dir / 'gdpnow_node2.txt', node)
    write_txt(text_dir / 'nowcast_text.txt', '\color{white}{.}')
    latest_txt = ''
    mark = ''
            
text = (f'The {latest_txt}nowcast for {ltdt} is {lvt} percent, as '+
        f'of {dtxt(pd.to_datetime(update))["day1"]}{mark}.')
write_txt(text_dir / 'gdpnow.txt', text)
print(text)

The latest nowcast for 2023 Q4 is 2.3 percent, as of October 27, 2023 (see \tikz \draw[black, fill=red] (2.5pt,2.5pt) circle (2.5pt);).


### Monthly nominal GDP measure

Does not show meaningful differences within months, basic interpolation. Adds latest quarter from nowcasts.

In [5]:
pce = pd.read_csv(data_dir / 'pce_now.csv', index_col='date', parse_dates=True)
gdp = gdpnow.copy().astype('float')
gdp.index = pd.to_datetime(gdp.index)
data = pd.concat([gdp, pce], axis=1)
data['real'] = data[['value', 'pce']].sum(axis=1)

In [6]:
df = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
mult = ((data['real'].iloc[0] / 4) / 100) + 1
df.loc[data.index[0]] = df.iloc[-1] * mult
df.index = (df.index + pd.DateOffset(months=2)).to_period('M')
mon = df.resample('M').interpolate()
mon.index = mon.index.to_timestamp()
mon.to_csv(data_dir / 'gdp_monthly.csv', index_label='date')

### Estimate of labor productivity

In [35]:
data = pd.read_csv(data_dir / 'gdpjobslvl.csv', 
                   index_col='date', parse_dates=True)
currdt = data['GDP'].dropna().index[-1]
gdpnow = pd.read_csv(data_dir / 'gdpnow.csv', index_col='date', parse_dates=True)

qch = ((gdpnow['value'].iloc[0] / 100) + 1) ** (1/4)
nowdt = gdpnow.index[0]
prdt = nowdt - pd.DateOffset(months=3)

if pd.isna(data.loc[nowdt, 'GDP']):
    if pd.isna(data.loc[nowdt, 'TOT_HRS']):
        popch = (data['POP'].pct_change().iloc[-1]) + 1
        data.loc[nowdt, 'TOT_HRS'] = data.loc[prdt, 'TOT_HRS'] * popch
        data.loc[nowdt, 'EPOP_sa'] = data.loc[prdt, 'EPOP_sa']
        data.loc[nowdt, 'AAH_trend'] = data.loc[prdt, 'AAH_trend']
    data.loc[nowdt, 'GDP'] = data.loc[prdt, 'GDP'] * qch
    data.loc[nowdt, 'LPROD'] = (data.loc[nowdt, 'GDP'] / 
                                data.loc[nowdt, 'TOT_HRS']) 

# Convert to indices
for i in ['LPROD','TOT_HRS', 'GDP', 
          'POP', 'EPOP_sa', 'AAH_trend']:
    data[f'{i}_ix'] = (data[i] / data.loc['2000-01-01', i]) * 100

# Calculate productivity growth rate
data['LPROD_gr'] = (((data.LPROD.dropna().pct_change() + 1) ** 4) - 1) * 100
    
# Create table
cl = c_line('cyan!70!white', see=False, paren=False)
names = {'LPROD_ix': f'\hspace{{0.1mm}}{cl} Labor Productivity (index)', 
         'GDP_ix': '\hspace{4mm} Real GDP (index)',
         'TOT_HRS_ix': '\hspace{4mm} Total Hours Worked (index)', 
         'POP': '\hspace{7mm} Population (millions)', 
         'EPOP_sa': '\hspace{7mm} Employment Rate (percent)', 
         'AAH_trend': '\hspace{7mm} Average Workweek (hours)',
         'LPROD_gr': '\hspace{0.1mm} Labor Productivity Growth (percent)'}

table = data[names.keys()].dropna().iloc[[-1,-2,-3,-5]].T
table['2019-10-01'] = data.loc['2019-10-01', names.keys()]
table.columns = [dtxt(c)['qtr1'] for c in table.columns]
table['2014'] = data.loc['2014', names.keys()].mean()
table['1989'] = data.loc['1989', names.keys()].mean()
table = table.rename(names)
nowqt = dtxt(nowdt)['qtr1']
if nowdt > currdt:
    table[nowqt] = (table[nowqt].apply('\\textit{{{:.1f}}}'.format))
    table.iloc[:, 1:] = table.iloc[:, 1:].applymap('{:.1f}'.format)
    table.columns = table.columns.str.replace(nowqt, f'\\\\textit{{{{{nowqt}}} Est.}}')
else:
    table = table.applymap('{:.1f}'.format)
table.to_csv(data_dir / 'gdpjobs.tex', sep='&', 
             lineterminator='\\\ ', quotechar=' ')

In [36]:
# Only add mark to plot if nowcast is available
mark = ' (see \\tikz \draw[black, fill=cyan!70!white] (2.5pt,2.5pt) circle (2.5pt);)'
prval = data.loc[currdt, 'LPROD_gr']
nowval = data.loc[nowdt, 'LPROD_gr']
lv = data.loc[nowdt, 'LPROD_ix']
lvt = f'{lv:.1f}'
cht = f'{nowval:.1f}'
if currdt < nowdt:
    node = (f'\\node[label={{[align=left]90:{{\scriptsize \\textit{{{lvt}}}}}}}, '+
            'circle, draw=black, fill=cyan!70!white, inner sep=1.2pt] at '+
            f'(axis cs:{nowdt.date()}, {lv}) {{}};')
    node2 = (f'\\node[label={{[align=left]90:{{\scriptsize \\textit{{{cht}}}}}}}, '+
            'circle, draw=black, fill=cyan!70!white, inner sep=1.2pt] at '+
            f'(axis cs:{nowdt.date()}, {nowval}) {{}};')
    txt = (f'The estimate for {nowqt}{mark}, based on the Federal Reserve Bank of '+
        'Atlanta GDPNow, suggests annualized productivity growth of '+
        f'{nowval:.1f} percent.')
    write_txt(text_dir / 'gdpjobs_est_node.txt', node)
    write_txt(text_dir / 'lprod_rec_node.txt', node2)
else:
    node = ''
    txt = ''
    write_txt(text_dir / 'gdpjobs_est_node.txt', node)

text = ('More-recent data show annualized total economy productivity '+
        f'growth of {prval:.1f} percent in {dtxt(currdt)["qtr1"]}. {txt}')
write_txt(text_dir / 'gdpjobs_est.txt', text)
print(text)

More-recent data show annualized total economy productivity growth of 4.2 percent in 2023 Q3. The estimate for 2023 Q4 (see \tikz \draw[black, fill=cyan!70!white] (2.5pt,2.5pt) circle (2.5pt);), based on the Federal Reserve Bank of Atlanta GDPNow, suggests annualized productivity growth of 1.8 percent.


In [37]:
rec = data.loc[:currdt, 'LPROD_gr'].iloc[-8:].to_frame()
rec['Quarter'] = [f'Q{i.quarter}\\\\{i.year}' if i.quarter == 1 
                  else dtxt(i)['qtr3'] for i in rec.index]
# From GDP Now (cover cases where no nowcast available)
if currdt < nowdt:
    next_qtr = rec.index[-1] + pd.DateOffset(months=3)
    label = (f'Q{next_qtr.quarter}\\\\{next_qtr.year}' 
             if next_qtr.quarter == 1 else dtxt(next_qtr)['qtr3'])
    frow = pd.DataFrame({'Quarter': label, 
                         'date': next_qtr}, index={'date': 4})
    rec= pd.concat([rec.reset_index(), frow]).set_index('date')
    rec.loc[next_qtr, 'LPROD_gr'] = 0
rec['zero'] = 0
rec.to_csv(data_dir / 'lprod_rec.csv', index_label='date')

# Average bar
start_date = rec.index[0] - pd.DateOffset(months=2)
end_date = rec.index[-1] + pd.DateOffset(months=1)
val = data['LPROD_gr'].iloc[:-1].astype('float').mean()
color = 'gray!60!white'
bar = (f'\draw [{color}] (axis cs:{{{start_date}}},{val}) -- '+
       f'(axis cs:{{{end_date}}},{val});')
bardf = pd.Series(index=[start_date, end_date], 
                data=[val, val], name='Bar')
node = end_node(bardf, color, loc='start')
write_txt(text_dir / 'lprod_rec_bar_node.txt', bar + '\n' + node)

### GDP Intro Chart and Text

In [38]:
df = nipa_df(retrieve_table('T10502')['Data'],
             ['A191RL'])['A191RL']
# Dates for latest quarters chart
a = df.iloc[-8:]
a['Quarter'] = [f'Q{i.quarter}\\\\{i.year}' if i.quarter == 1 
                else dtxt(i)['qtr3'] for i in a.index]
# From GDP Now (cover cases where no nowcast available)
if gdpdt < nowdt:
    next_qtr = rec.index[-1] + pd.DateOffset(months=3)
    label = (f'Q{next_qtr.quarter}\\\\{next_qtr.year}' 
             if next_qtr.quarter == 1 else dtxt(next_qtr)['qtr3'])
    frow = pd.DataFrame({'Quarter': label, 
                         'index': next_qtr}, index={'index': 4})
    rec= pd.concat([rec.reset_index(), frow]).set_index('index')
    rec.loc[next_qtr, 'A191RL'] = 0
rec['zero'] = 0
a.to_csv(data_dir/'gdp_rec2.csv', index_label='date')

In [39]:
df = nipa_df(retrieve_table('T70100')['Data'], ['A939RC', 'A939RX'])
df['value'] = (df['A939RX'] / df['A939RX'].iloc[-1]) * df['A939RC'].iloc[-1]
df.loc['1989':, 'value'].divide(1000).to_csv(data_dir / 'gdppc.csv', 
                                             index_label='date')
cd = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC'].iloc[-1]
rgdp = nipa_df(retrieve_table('T10106')['Data'], ['A191RX'])
rgdp_cd = rgdp / rgdp.iloc[-1] * cd

ltdate = dtxt(df.index[-1])['qtr2']
gdp_val = int(rgdp_cd.iloc[-1].values[0] / 1000)
gdp_prv_val = int(rgdp_cd.loc['2019-10-01'] / 1000)
gdp_prv_val2 = int(rgdp_cd.iloc[0].values[0] / 1000)
gdppc_val = int(df.iloc[-1].loc['value'])
gdppc_prv_val = int(df.loc['2019-10-01', 'value'])
gdppc_prv_val2 = int(df.iloc[0].loc['value'])
qdate = dtxt(df.index[-1])['qtr1']
pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']
num = numbers[f'{(pop.pct_change(4).iloc[-1] * 1000).round()}']
popnum = f'{num}-tenths of' if num != 'five' else 'half'
color = 'red!95!black'
cl = c_line(color)
node = end_node(df['value'] / 1000, color, anchor='south', 
                dollar='thousands')
write_txt(text_dir / 'gdp_pc_node.txt', node)
text = (f'\${gdp_val:,} billion in {ltdate}, compared to an '+
        f'inflation-adjusted equivalent of \${gdp_prv_val:,} '+
        f'billion in 2019 Q4, and \${gdp_prv_val2:,} billion in '+
        'the first quarter of 1989.\n\nThe US population is growing '+
        f'by about {popnum} a percent per year. GDP per capita '+
        f'{cl}, adjusted for inflation to {qdate} dollars, had '+
        f'increased to \${gdppc_prv_val:,} in 2019 Q4 from '+
        f'\${gdppc_prv_val2:,} in 1989 Q1, and is currently '+
        f'\${gdppc_val:,}, as of {ltdate}.')
write_txt(text_dir / 'gdp1.txt', text)
print(text)

\$27,623 billion in the third quarter of 2023, compared to an inflation-adjusted equivalent of \$25,731 billion in 2019 Q4, and \$11,504 billion in the first quarter of 1989.

The US population is growing by about half a percent per year. GDP per capita (see {\color{red!95!black}\textbf{---}}), adjusted for inflation to 2023 Q3 dollars, had increased to \$77,690 in 2019 Q4 from \$47,111 in 1989 Q1, and is currently \$82,335, as of the third quarter of 2023.


### GDP components

In [40]:
pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']

srs = {'A191RX': '\hspace{0.01mm} {\color{red!95!black}\\textbf{---}} Gross Domestic Product', 
       'DPCERX': '\hspace{1.0mm} {\color{yellow!45!orange}\\textbf{---}} Consumer Spending', 
       'A006RX': '\hspace{1.0mm} {\color{blue!70!black}\\textbf{---}} Gross Private Domestic Investment', 
       'A822RX': '\hspace{1.0mm} {\color{cyan!60!white}\\textbf{---}} Government Spending \& Investment',
       'A019RX': '\hspace{1.0mm} {\color{green!60!black}\\textbf{---}} Net Exports', 
       'A020RX': '\hspace{4.5mm} Exports', 'A021RX': '\hspace{4.5mm} Less: Imports'}
s = [key for key, value in srs.items()]
s2 = [i[:-1].replace('A02', 'B02') + 'C' for i in s]
df = nipa_df(retrieve_table('T10106')['Data'], s).sort_index()
df2 = nipa_df(retrieve_table('T10105')['Data'], s2).sort_index()

dfpop = df.div(pop, axis=0)
real_vals = df2.rename({i: i.replace('RC', 'RX').replace('B02', 'A02') 
                        for i in s2}, axis=1).iloc[-1]
data = ((dfpop / dfpop.iloc[-1]) * (real_vals / pop.iloc[-1])).loc['1989':]

sel_cols = ['DPCERX', 'A006RX', 'A019RX', 'A822RX']

data[sel_cols].to_csv(data_dir / 'gdp_pc_comp.csv', index_label='date')

pce = f'\${data.DPCERX.iloc[-1] * 1000:,.0f}'
inv = f'\${data.A006RX.iloc[-1] * 1000:,.0f}'
gov = f'\${data.A822RX.iloc[-1] * 1000:,.0f}'
exp = f'\${abs(data.A019RX.iloc[-1]) * 1000:,.0f}'
ltdt = dtxt(data.index[-1])['qtr1']
pce_diff = f'\${(data.DPCERX.iloc[-1] - data.DPCERX.iloc[0]) * 1000:,.0f}'

text = ('Much of the increase in GDP over the past '+
        '30 years comes from consumer spending. Consumer '+
        'spending (see {\color{yellow!45!orange}\\textbf{---}}) is '+
        f'equivalent to {pce} per person in {ltdt}, a price-adjusted '+
        f'increase of {pce_diff} since 1989.\n\nGross private domestic '+
        'investment (see {\color{blue!70!black}\\textbf{---}}) is '+
        f'equivalent to {inv} per person in {ltdt}. Government '+
        'spending and investment (see {\color{cyan!60!white}\\textbf{---}}) '+
        f'totals {gov} per person. The trade deficit, equivalent to {exp} per '+
        'person, is subtracted, to capture only domestic production '+
        '(see {\color{green!60!black}\\textbf{---}}).')

write_txt(text_dir / 'gdp_pc_comp.txt', text)
write_txt(text_dir / 'gdp_ltdt.txt', ltdt)
print(text)

res = data * 1000
lt = res.rename(srs, axis=1).iloc[-1]
lt.name = dtxt(lt.name)['qtr1']
pr = res.rename(srs, axis=1).iloc[-2]
pr.name = dtxt(pr.name)['qtr1']
pc = res.rename(srs, axis=1).loc['2019-10-01']
pc.name = dtxt(pc.name)['qtr1']
p00 = res.rename(srs, axis=1).loc['2000-01-01']
p00.name = dtxt(p00.name)['qtr1']
fi = res.rename(srs, axis=1).iloc[0]
fi.name = dtxt(fi.name)['qtr1']
table = pd.concat([lt, pr, pc, p00, fi], axis=1).applymap('{:,.0f}'.format)
table.iloc[0, 0] = f'\${table.iloc[0, 0]}'
table.to_csv(data_dir / 'gdppc_levels.tex', sep='&', 
             lineterminator='\\\ ', quotechar=' ')

Much of the increase in GDP over the past 30 years comes from consumer spending. Consumer spending (see {\color{yellow!45!orange}\textbf{---}}) is equivalent to \$55,840 per person in 2023 Q3, a price-adjusted increase of \$25,483 since 1989.

Gross private domestic investment (see {\color{blue!70!black}\textbf{---}}) is equivalent to \$14,603 per person in 2023 Q3. Government spending and investment (see {\color{cyan!60!white}\textbf{---}}) totals \$14,231 per person. The trade deficit, equivalent to \$2,339 per person, is subtracted, to capture only domestic production (see {\color{green!60!black}\textbf{---}}).


### Labor Share

In [16]:
# Compensation of employees divided by gross domestic income less 
# depreciation (net domestic income, as depreciation is income that
# does not go to people)
s = ['A261RC', 'A4002C', 'A262RC']

df = nipa_df(retrieve_table('T11000')['Data'], s).sort_index()
df['NDI'] = (df['A261RC'] - df['A262RC'])
df['Share'] = (df['A4002C'] / df['NDI']) * 100
data = df.loc['1989':, 'Share'].dropna()
data.to_csv(data_dir / 'laborshare.csv', 
            index_label='date', 
            float_format='%g')

node = end_node(data, 'blue!60!cyan', date='q')
write_txt(text_dir / 'laborshare_node.txt', node)

s = series_info(data)
ltdate = dtxt(s['date_latest'])['qtr2']
one_yr = value_text(s['val_latest'] - s['val_year_ago'], ptype='pp')
ltmin = s['val_latest'] - s['val_min']
ltmax = s['val_max'] - s['val_latest']

# Note on value of labor share
emp = pd.read_csv(data_dir / 'ces_raw.csv', index_col='date', 
                  parse_dates=True)['CES0000000001'] / 1_000
emp = emp.resample('QS').mean()
tot = ((df['NDI'].dropna()) * 0.01)
ltndi = tot.iloc[-1] 
ltemp = emp.loc[tot.index[-1]]
per_emp = (ltndi / ltemp).round(-1)

ltdt = dtxt(tot.index[-1])['qtr1']

ctxt = ('For context, one percent of annual NDI is equivalent to '+
        f'\${tot.iloc[-1] / 1000:.0f} billion, or '+
        f'\${per_emp:,.0f} per worker. ')

text = (f'As of {ltdate}, labor receives {s["val_latest"]:.1f} '+
        f"percent of NDI. Labor's share {one_yr} "+
        f'over the past year. {ctxt}\n\nThe labor share is {ltmin:.1f} '+
        f'percentage points above its 30-year low of {s["val_min"]:.1f} '+
        f'percent in {s["date_min_ft"]}, and {ltmax:.1f} percentage '+
        f'points below the 30-year high of {s["val_max"]:.1f} percent '+
        f'in {s["date_max_ft"]}. ')
write_txt(text_dir / 'laborshare.txt', text)
print(text)

As of the second quarter of 2023, labor receives 64.2 percent of NDI. Labor's share increased 2.1 percentage points over the past year. For context, one percent of annual NDI is equivalent to \$221 billion, or \$1,420 per worker. 

The labor share is 2.7 percentage points above its 30-year low of 61.6 percent in 2014 Q3, and 4.0 percentage points below the 30-year high of 68.3 percent in 2020 Q2. 


### GDP growth rate

In [17]:
s = ['A191RL']
df = nipa_df(retrieve_table('T10502')['Data'], s).sort_index()
df.loc['1989':].to_csv(data_dir / 'gdp.csv', index_label='date')
date = dtxt(df.index[-1])['qtr1']

txt = f'{date}: {df["A191RL"].iloc[-1]}\%'
write_txt(data_dir / 'gdp.txt', txt)

In [18]:
dfa = (nipa_df(retrieve_table('T10101A')['Data'], ['A191RL'])
       ['A191RL'])
dfa.index = dfa.index + pd.DateOffset(months=6)
dfa.loc['1989':].to_csv(data_dir / 'gdp_a.csv', index_label='date')

In [44]:
a

,A191RL,zero,Quarter
index,,,
2022-10-01,2.6,0,Q4
2023-01-01,2.2,0,Q1\\2023
2023-04-01,2.1,0,Q2
2023-07-01,4.9,0,Q3
2023-10-01,0.0,0,Q4


In [43]:
df = nipa_df(retrieve_table('T10502')['Data'],
             ['A191RL'])['A191RL']
# Dates for latest quarters chart
a = df.iloc[-4:].to_frame()
next_qtr = a.index[-1] + pd.DateOffset(months=3)
frow = pd.DataFrame({'index': next_qtr}, index={'index': 4})
# From GDP Now (cover cases where no nowcast available)
if gdpdt < nowdt:
    a = pd.concat([a.reset_index(), frow]).set_index('index')
    a.loc[next_qtr, 'A191RL'] = 0
a['zero'] = 0
a['Quarter'] = [f'Q{i.quarter}\\\\{i.year}' if i.quarter == 1 
                  else dtxt(i)['qtr3'] for i in a.index]
a.to_csv(data_dir/'gdp_rec.csv', index_label='date')
# Text
lty = df.index[-1].year
ltdt = dtxt(df.index[-1])['qtr2']
ltdt2 = dtxt(df.index[-1])['qtr1']
pry = df.index[-2].year
pr2y = df.index[-3].year
prdt = dtxt(df.index[-2])
prdt = prdt['qtr3'] if lty == pry else prdt['qtr1']
pr2dt = dtxt(df.index[-3])
pr2dt = pr2dt['qtr3'] if lty == pr2y else pr2dt['qtr1']
gtot = df.loc['1989':].mean()
g9300 = df.loc['1993':'2000'].mean()
g0013 = df.loc['2001':'2013'].mean()
g1419 = df.loc['2014':'2019'].mean()
# expenditure approach data
d19 = (nipa_df(retrieve_table('T10106')['Data'], ['A191RX'])
       .loc['2019-10-01':, 'A191RX'])
g20on = cagr(d19)
cov1 = value_text(df.loc['2020-04-01'])
cov2 = value_text(df.loc['2020-07-01'], 'increase_by')
ltval = value_text(df.iloc[-1], adj='annual')
prval = value_text(df.iloc[-2], 'increase_of')
prval2 = value_text(df.iloc[-3], 'increase_of')

color='red'
url = 'https://www.bea.gov/data/gdp/gross-domestic-product'
text = (f'Real GDP growth \href{{{url}}}{{measures}} changes in '+
        'economic activity. As seen in the previous subsection, '+
        'real GDP has increased steadily over the long-term. '+
        'Since 1989, growth averaged '+
        f'{gtot:.1f} percent per year {c_box(color)}. '+
        'Growth rates were relatively high during the mid- '+
        f'to late-1990s, averaging {g9300:.1f} percent '+
        'from 1993 to 2000.\n\nIn the 2000s, the housing '+
        'bubble boosted GDP but then collapsed, '+
        f'leading to average growth of only {g0013:.1f} '+
        'percent from 2001 to 2013. Growth was slightly '+
        f'stronger from 2014 to 2019, averaging {g1419:.1f} '+
        'percent per year.\n\nIn 2020, COVID-19 caused '+
        'an economic shutdown, followed by monetary '+
        'and fiscal stimulus, resulting in large swings in '+
        f'GDP. Annualized real GDP {cov1} in Q2, and {cov2} in Q3, '+
        'by far the largest changes in recent history. Since 2019 Q4, '+
        f'real GDP has grown at an average annual rate of {g20on:.1f} '+
        'percent.')
write_txt(text_dir / 'gdp_gr.txt', text)
print(text)

txt2 = ('The bottom-left chart shows annual growth, to make '+
        'trends more visible. The bottom-right chart shows '+
        'the most-recent four quarters and the estimate for the '+
        'current quarter. In the \\textbf{latest data}, '+
        f'covering {ltdt}, real GDP {ltval}, compared to {prval} '+
        f'in {prdt}, and {prval2} in {pr2dt}.')
write_txt(text_dir / 'gdp_gr2.txt', txt2)
print('\n', txt2)

Real GDP growth \href{https://www.bea.gov/data/gdp/gross-domestic-product}{measures} changes in economic activity. As seen in the previous subsection, real GDP has increased steadily over the long-term. Since 1989, growth averaged 2.6 percent per year (see\cbox{red}). Growth rates were relatively high during the mid- to late-1990s, averaging 3.8 percent from 1993 to 2000.

In the 2000s, the housing bubble boosted GDP but then collapsed, leading to average growth of only 1.9 percent from 2001 to 2013. Growth was slightly stronger from 2014 to 2019, averaging 2.6 percent per year.

In 2020, COVID-19 caused an economic shutdown, followed by monetary and fiscal stimulus, resulting in large swings in GDP. Annualized real GDP decreased 28.0 percent in Q2, and increased by 34.8 percent in Q3, by far the largest changes in recent history. Since 2019 Q4, real GDP has grown at an average annual rate of 1.9 percent.

 The bottom-left chart shows annual growth, to make trends more visible. The bot

In [45]:
# Dates for latest quarters chart
a = df.iloc[-8:].to_frame()
a['Quarter'] = [f'Q{i.quarter}\\\\{i.year}' if i.quarter == 1 
                  else dtxt(i)['qtr3'] for i in a.index]
# From GDP Now (cover cases where no nowcast available)
if gdpdt < nowdt:
    next_qtr = a.index[-1] + pd.DateOffset(months=3)
    label = (f'Q{next_qtr.quarter}\\\\{next_qtr.year}' 
             if next_qtr.quarter == 1 else dtxt(next_qtr)['qtr3'])
    frow = pd.DataFrame({'Quarter': label, 
                         'index': next_qtr}, index={'index': 4})
    a = pd.concat([a.reset_index(), frow]).set_index('index')
    a.loc[next_qtr, 'A191RL'] = 0
a['zero'] = 0
a.to_csv(data_dir/'gdp_rec2.csv', index_label='date')

# Average bar
start_date = a.index[0] - pd.DateOffset(months=2)
end_date = a.index[-1] + pd.DateOffset(months=1)
val = df.mean()
color = 'gray!60!white'
bar = (f'\draw [{color}] (axis cs:{{{start_date}}},{val}) -- '+
       f'(axis cs:{{{end_date}}},{val});')
bardf = pd.Series(index=[start_date, end_date], 
                data=[val, val], name='Bar')
node = end_node(bardf, color, loc='start')
write_txt(text_dir / 'gdp_rec2_bar_node.txt', bar + '\n' + node)

### Nominal GDP decomposed as Real GDP and Prices

In [21]:
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
df = nipa_df(retrieve_table('T10502')['Data'], ['A191RL'])
gdpgr = growth_rate(gdp)
df['PR'] = growth_rate(gdp) - df['A191RL']
df.loc['1989':].to_csv(data_dir / 'ngdp_gr.csv', index_label='date')
(gdp.loc['1989':] / 1_000_000).to_csv(data_dir / 'ngdp.csv', 
                                      index_label='date')

In [22]:
ltdt = dtxt(gdpgr.index[-1])['qtr2']
prdt = dtxt(gdpgr.index[-2])['qtr1']
pr2dt = dtxt(gdpgr.index[-3])['qtr1']
ltch = value_text(gdpgr.iloc[-1], adj='annual')
prch = value_text(gdpgr.iloc[-2], 'increase_of')
pr2ch = value_text(gdpgr.iloc[-3], 'increase_of')

ann19 = value_text(cagr(gdp.loc['2019-10-01':]), 'plain')
ann89 = value_text(cagr(gdp.loc['1989-01-01':'2019-10-01']), adj='annual')
    
text = (f'In {ltdt}, nominal GDP {ltch}, following {prch} in {prdt}, '+
        f'and {pr2ch} in {pr2dt}. From 1989 to 2019 Q4, nominal GDP '+
        f'{ann89}. Since 2019 Q4, the annualized growth rate is {ann19}.')
write_txt(text_dir / 'ngdp_gr.txt', text)
print(text)

In the third quarter of 2023, nominal GDP increased at an annual rate of 8.5 percent, following an increase of 3.8 percent in 2023 Q2, and an increase of 6.3 percent in 2023 Q1. From 1989 to 2019 Q4, nominal GDP increased at an annual rate of 4.6 percent. Since 2019 Q4, the annualized growth rate is 6.4 percent.


### Recessions Table

In [23]:
rec = fred_df('USREC')
rec.to_csv(data_dir / 'recessions_raw.csv', index_label='date')

In [24]:
rec = pd.read_csv(data_dir / 'recessions_raw.csv', 
                  index_col='date', parse_dates=True)
first = rec[(rec.VALUE==1) & (rec.VALUE.shift(1) == 0)]
post = rec[(rec.VALUE==0) & (rec.VALUE.shift(1) == 1)]
names = [' \ Early `90s Recession', ' \ Early `00s Recession', 
         ' \ Great Recession', ' \ COVID-19 Recession']
recs = (pd.Series(data=first.index, index=names)
        .rename('First').to_frame())
recs['Last'] = rec[(rec.VALUE==1) & (rec.VALUE.shift(-1) == 0)].index
recs['Pre'] = rec[(rec.VALUE==0) & (rec.VALUE.shift(-1) == 1)].index
recs['Post'] = post.index
dur = [i.n for i in (post.index.to_period('M') - 
                     first.index.to_period('M'))]
recs['Dur'] = pd.Series(data=dur, index=recs.index)
recs['PrevEnd'] = recs['Post'].shift(1)
recs.loc[' \ Early `90s Recession', 'PrevEnd'] = pd.to_datetime('1989-01-01')
recs['NextStart'] = recs['Pre'].shift(-1)
recs.loc[' \ COVID-19 Recession', 'NextStart'] = cps_date()
recs['Start'] = recs.First.apply(lambda x: dtxt(x)['mon2'])
recs['End'] = recs.Last.apply(lambda x: dtxt(x)['mon2'])
rgdp = nipa_df(retrieve_table('T10106')['Data'], ['A191RX'])['A191RX']
unrate = pd.read_csv(data_dir / 'jobs_report_main.csv', index_col='date', 
                 parse_dates=True)['Total']
for row in recs.itertuples():
    # Real GDP change
    vprev = rgdp.loc[:row.Pre].max()
    vmin = rgdp.loc[row.First:row.NextStart].min()
    ch = ((vmin / vprev) - 1) * 100
    recs.loc[row.Index, 'GDPch'] = ch
    # Unemployment rate change and duration
    pravg = unrate.loc[row.Pre - pd.DateOffset(years=3): row.Pre].mean()
    vmax = unrate.loc[row.First:row.NextStart].max()
    uch = vmax - pravg
    rdt = (unrate.loc[row.Last:].loc[(unrate <= pravg)].index[0] 
           if pravg >= unrate.iloc[-1] else '--')
    rtime = (int((rdt.to_period('M') - row.Last.to_period('M')).n) 
             if rdt != '--' else '--')
    recs.loc[row.Index, 'Unratech'] = uch    
    recs.loc[row.Index, 'RecoDate'] = rdt
    recs.loc[row.Index, 'RecoTime'] = str(rtime)
recs['GDPcht'] =  recs.GDPch.apply('{:.1f}'.format)
recs['Uncht'] =  recs.Unratech.apply('+{:.1f}'.format)
tbl = recs[['Start', 'End', 'Dur', 'GDPcht', 'Uncht', 'RecoTime']]
tbl.columns = ['Start \ \ \ Month', 'End \ \ \ \ \ \ Month', 
               'Recession Duration, Months', 
               'GDP Percent Change', 'Unemp. Rate Change*', 
               'Unemp. Rate Recovery, Months**']
tbl.to_csv(data_dir / 'recession.tex', sep='&', 
           lineterminator='\\\ ', quotechar=' ')

un3 = unrate.rolling(3).mean()
sahm = (un3 - un3.rolling(12).min()).dropna()
sahm.to_csv(data_dir / 'sahm.csv', index_label='date', 
            header=True)
bar = pd.Series(index=[sahm.index[0], sahm.index[-1]], 
                data=[0.5, 0.5], name='Bar')
bar.to_csv(data_dir / 'sahm_bar.csv', index_label='date', 
           header=True)
node = end_node(bar, 'gray', loc='start')
write_txt(text_dir / 'sahm_bar_node.txt', node)
marks = (sahm.loc[(sahm > 0.5) & (sahm.shift(1) < 0.5)]
             .rename('Mark').to_frame())
marks['Intersect'] = len(marks) * [0.5]
marks.to_csv(data_dir / 'sahm_marks.csv', index_label='date')

dur90 = numbers[f'{recs.Dur.iloc[0]:.1f}']
unrec90 = recs.RecoTime.iloc[0]
unrec00 = round(int(recs.RecoTime.iloc[1]) / 12)
durgr = recs.Dur.iloc[2]
unrecgr = recs.RecoTime.iloc[2]
durco = numbers[f'{recs.Dur.iloc[3]:.1f}']
gdpco = abs(recs.GDPch.iloc[3])

text = ('During the early 1990s recession, output contracted '+
        f'for {dur90} months and unemployment was higher '+
        f'than its pre-recession average for {unrec90} months. '+
        'The drop in output was smaller during the '+
        'early 2000s recession, but unemployment rates '+
        f'took almost {unrec00} years to recover.\n\n'+
        'The 2008--2009 great recession, caused by the '+
        'collapse of a housing bubble, was very severe. '+
        f'The recession lasted {durgr} months, with higher '+
        f'rates of unemployment lasting {unrecgr} months. The '+
        'most-recent COVID-19 recession was extremely severe '+
        f'and also extremely short-lived, lasting only {durco} '+
        f'months, but with output reduced {gdpco:.1f} percent.')
write_txt(text_dir / 'recessions.txt', text)
print(text)

During the early 1990s recession, output contracted for eight months and unemployment was higher than its pre-recession average for 63 months. The drop in output was smaller during the early 2000s recession, but unemployment rates took almost 16 years to recover.

The 2008--2009 great recession, caused by the collapse of a housing bubble, was very severe. The recession lasted 18 months, with higher rates of unemployment lasting 89 months. The most-recent COVID-19 recession was extremely severe and also extremely short-lived, lasting only two months, but with output reduced 9.1 percent.


### Investment overview

In [21]:
#nipa_series_codes(retrieve_table('T10105'))

In [25]:
d = {'T10105': {'A007RC': 'PrGrossFixed', 'A008RC': 'PrGrossFixedNR', 
                'A011RC': 'PrGrossFixedRes', 'A191RC': 'GDP', 
                'A014RC': 'CIPI'},
     'T30100': {'A782RC': 'GovGross'}}

df = pd.DataFrame()
for t, s in d.items():
    i = nipa_df(retrieve_table(t)['Data'], s.keys()).rename(d, axis=1)
    for v, n in s.items():
        df[n] = i[v]
        
# Sum public and private gross investment
df['Gross'] = df['PrGrossFixed'] + df['GovGross']
df.to_csv(data_dir / 'grossinv_lvl.csv', index_label='date')

# Share of GDP
sh = df.divide(df.GDP, axis=0) * 100
sh.loc['1989':].to_csv(data_dir / 'grossinv_sh.csv', index_label='date')

# Text 
ltdt = dtxt(sh.index[-1])['qtr2']
ltval = f'\${df.Gross.iloc[-1] / 1_000_000:.1f} trillion'
ltsh = f'{sh.Gross.iloc[-1]:.1f} percent of GDP'
prsh = f'{sh.Gross.iloc[-5]:.1f} percent of GDP'
prdt = dtxt(sh.index[-5])['qtr1'] 
sh19 = f'{sh.loc["2019", "Gross"].mean():.1f} percent of GDP'
also = 'also ' if prsh == sh19 else ''
cl = c_line('black')

text = (f'In {ltdt}, annualized US \\textbf{{gross fixed investment}}, '+
        f'both public and private, totals {ltval}, or {ltsh} '+
        f'{cl}. Gross fixed investment is equivalent to {prsh} '+
        f'one year prior, in {prdt}, and {also}averages {sh19} '+
        'in 2019. ')
write_txt(text_dir / 'gross_inv_total.txt', text)
print(text, '\n')

# Node for total
node = end_node(sh.Gross, 'black', date='qs', offset=0.2)
write_txt(text_dir / 'gross_inv_total_node.txt', node)
# Nodes for latest values
cols = ['GovGross', 'PrGrossFixedRes', 'PrGrossFixedNR']
sdf = sh[cols].iloc[-1]
height = ((sdf.cumsum() - (sdf / 2) + 1.5)).to_dict()
val = sdf.to_dict()
dtp = dtxt(sh.index[-1] + pd.DateOffset(months=2))['datetime']
nodes = [f'\\absnode{{{{{dtp}}}}}{{{height[i]}}}{{\scriptsize {val[i]:.1f}}}' 
         for i in cols]
nodetext = '\n'.join(nodes)
write_txt(text_dir / 'gross_inv_nodes.txt', nodetext)

# Color boxes in text
govcb = c_box('violet!60!black').replace('see ', 'see')
rescb = c_box('blue!60!white').replace('see ', 'see')
nrcb = c_box('yellow!70!orange!90!white').replace('see ', 'see')

# Text 2
ltdt = dtxt(sh.index[-1])['qtr1']
st = sh.iloc[-1].to_frame()
st['sh'] = (st / st.loc['Gross']) * 100
st['sh_txt'] = st['sh'].apply(value_text, style='plain', digits=0)
st['gdp_txt'] = st.iloc[:,0].apply(value_text, style='plain')

text = (f'In {ltdt}, private nonresidential (business) fixed investment '+
        f'comprises {st.loc["PrGrossFixedNR", "sh_txt"]} of the '+
        f'total and is equivalent to {st.loc["PrGrossFixedNR", "gdp_txt"]} '+
        f'of GDP {nrcb}. Private residential makes up '+
        f'{st.loc["PrGrossFixedRes", "sh_txt"]} of the total and '+
        f'{st.loc["PrGrossFixedRes", "gdp_txt"]} of GDP {rescb}. Public '+
        f'investment is {st.loc["GovGross", "sh_txt"]} of the total '+
        f'and {st.loc["GovGross", "gdp_txt"]} of GDP {govcb}.')
print(text)
write_txt(text_dir / 'gross_inv_sector.txt', text)

In the third quarter of 2023, annualized US \textbf{gross fixed investment}, both public and private, totals \$5.8 trillion, or 21.0 percent of GDP (see {\color{black}\textbf{---}}). Gross fixed investment is equivalent to 21.3 percent of GDP one year prior, in 2022 Q3, and averages 21.4 percent of GDP in 2019.  

In 2023 Q3, private nonresidential (business) fixed investment comprises 64 percent of the total and is equivalent to 13.5 percent of GDP (see\cbox{yellow!70!orange!90!white}). Private residential makes up 19 percent of the total and 3.9 percent of GDP (see\cbox{blue!60!white}). Public investment is 17 percent of the total and 3.6 percent of GDP (see\cbox{violet!60!black}).


In [24]:
#nipa_series_codes(retrieve_table('T10502'))

### Gross fixed investment

In [26]:
# Retrieve levels data 
lvl = pd.read_csv(data_dir / 'grossinv_lvl.csv', 
                  index_col='date', parse_dates=True)
# Contribution to one-year growth
ac = growth_contrib_ann(lvl, 'GDP')
act = value_text(ac.Gross.iloc[-1], 'contribution_to', 
                 ptype='pp', digits=2)

# Contribution to annualized quarterly growth
s = ['A008RY', 'A011RY', 'A014RY', 'A788RY', 'A798RY', 
     'A799RY', 'A007RY']
gc = nipa_df(retrieve_table('T10502')['Data'], s)
gc['Gov'] = gc[['A788RY', 'A798RY', 'A799RY']].sum(axis=1)
gc['Total'] = gc[['A007RY', 'Gov']].sum(axis=1)
gc.loc['1989':].to_csv(data_dir / 'inv.csv', index_label='date')

# Example of strong investment growth in late 1990s
ex = value_text(gc.loc['1996': '1999', 'Total'].mean(), 
                    'added_lost', adj='average', digits=2, ptype='pp')

# Generate text
ltdate = dtxt(gc.index[-1])['qtr2']
ltdt = dtxt(gc.index[-1])['qtr1']
prdt = dtxt(gc.index[-2])['qtr1']
prdate = dtxt(gc.index[-2])['qtr5']
pr_dt = prdate if gc.index[-1].year == gc.index[-2].year else prdt
tot = value_text(gc.Total.iloc[-1], 'contribution_to', ptype='pp', 
                 digits=2, threshold=0.01)
totpr = value_text(gc.Total.iloc[-2], 'contribution_of', ptype='point',
                 digits=2, threshold=0.01)

bus = value_text(gc.A008RY.iloc[-1], 'contribution_to', ptype='pp', 
                 digits=2, threshold=0.01)
res = value_text(gc.A011RY.iloc[-1], 'contribution', ptype='pp', 
                 digits=2, threshold=0.01)
pub = value_text(gc.Gov.iloc[-1], 'contribution', ptype='pp', 
                 digits=2, threshold=0.01)
cipi = value_text(gc.A014RY.iloc[-1], 'contribution_to', ptype='pp', 
                  digits=2, threshold=0.01)
cipipr = value_text(gc.A014RY.iloc[-2], 'contribution_of', ptype='pp', 
                  digits=2, threshold=0.01)
cipi_yr = value_text(ac.CIPI.iloc[-1], 'contribution', ptype='pp', 
                     digits=2, threshold=0.01)
nr_yr = value_text(ac.PrGrossFixedNR.iloc[-1], 'contribution', ptype='pp', 
                     digits=2, threshold=0.01)
res_yr = value_text(ac.PrGrossFixedRes.iloc[-1], 'contribution', 
                    ptype='point', digits=2, threshold=0.01)
pub_yr = value_text(ac.GovGross.iloc[-1], 'contribution', ptype='point', 
                     digits=2, threshold=0.01)

# Color boxes in text
cbpub = c_box('violet!60!black').replace('see ', 'see')
cbres = c_box('blue!60!white').replace('see ', 'see')
cbbus = c_box('yellow!70!orange!90!white').replace('see ', 'see')
cbcipi = c_box('red').replace('see ', 'see')

text = ('For example, from 1996 to 1999, gross domestic fixed '+
        f'investment {ex} to annual real GDP growth. \n\n'
        f'In {ltdate}, gross domestic fixed investment {tot} '+
        f'annualized real GDP growth, following {totpr} in {pr_dt}. '+
        f'Over the past year, gross fixed investment {act} real '+
        'GDP growth.\n\n'+
        f'In {ltdt}, by type of gross fixed investment, private '+
        f'nonresidential {bus} annualized real GDP growth {cbbus}, '+
        f'private residential {res} {cbres}, and public {pub} '+
        f'{cbpub}. Over the past year, nonresidential or business '+
        f'gross fixed investment {nr_yr}, residential {res_yr}, and '+
        f'public {pub_yr}.')
write_txt(text_dir / 'inv_contrib_text.txt', text)
print(text, '\n')

text = (f'In {ltdt}, changes in private inventories {cipi} '+
        f'annualized real GDP growth {cbcipi}, following {cipipr} '+
        f'in {prdt}. Over the past year, changes in private '+
        f'inventories {cipi_yr} from real GDP growth.')
write_txt(text_dir / 'cipi_contrib_text.txt', text)
print(text)

For example, from 1996 to 1999, gross domestic fixed investment added an average of 1.75 percentage points to annual real GDP growth. 

In the third quarter of 2023, gross domestic fixed investment contributed 0.40 percentage point to annualized real GDP growth, following a contribution of 1.35 points in the second quarter. Over the past year, gross fixed investment contributed 1.02 percentage point to real GDP growth.

In 2023 Q3, by type of gross fixed investment, private nonresidential did not contribute to annualized real GDP growth (see\cbox{yellow!70!orange!90!white}), private residential contributed 0.15 percentage point (see\cbox{blue!60!white}), and public contributed 0.25 percentage point (see\cbox{violet!60!black}). Over the past year, nonresidential or business gross fixed investment contributed 0.87 percentage point, residential subtracted 0.28 point, and public contributed 0.44 point. 

In 2023 Q3, changes in private inventories contributed 1.32 percentage points to annua

### Net Fixed Investment

In [27]:
d = {'W171RC': 'NetDomInv',  'W790RC': 'NetBusInv', 
     'W791RC': 'NetHHInv', 'A889RC': 'NetGovInv', 
     'A928RC': 'GrossInv_NIPA', 'W170RC': 'GrossInv', 
     'A262RC': 'CFC', 'W276RC': 'CFCBus', 'W279RC': 'CFCHH',
     'A264RC': 'CFCGov', 'W987RC': 'GrossBusInv', 
     'W988RC': 'GrossHHInv', 'A782RC': 'GrossGovInv'}
df = nipa_df(retrieve_table('T50100')['Data'], d.keys()).rename(d, axis=1)
df['GDP'] = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])
sh = df.divide(df['GDP'], axis=0) * 100
sh.loc['1989':].to_csv(data_dir / 'bea_nfi.csv', 
             index_label='date', float_format='%g')

### Imports and Exports

The 2023 comprehensive update split table 4.2.5. 

In [28]:
# Import and Export share of GDP
s = ['B020RC', 'B021RC', 'B648RC', 'LA000006']
s2 = ['A191RC']
df1 = nipa_df(retrieve_table('T40205A')['Data'], s)
df2 = nipa_df(retrieve_table('T40205B')['Data'], s)
df = pd.concat([df1, df2])
df['A191RC'] = nipa_df(retrieve_table('T10105')['Data'], s2)
df['EX'] = df['B020RC'] - df['LA000006']
df['IM'] = df['B021RC'] - df['B648RC']
data = (df.div(df['A191RC'], axis=0) * 100).loc['1989':]
data.to_csv(data_dir / 'eximgdp.csv', index_label='date')

In [29]:
data = pd.read_csv(data_dir / 'eximgdp.csv', index_col='date', 
                   parse_dates=True)
excol = 'green!60!teal!80!black'
imcol = 'blue!90!cyan'

node = end_node(data.IM, imcol, date='q', offset=0.35)
write_txt(text_dir / 'np_im_node.txt', node)
node = end_node(data.EX, excol)
write_txt(text_dir / 'np_ex_node.txt', node)

date = dtxt(data.index[-1])['qtr2']
pc_dt = '2019-10-01'
pc_date = dtxt(pd.to_datetime(pc_dt))['qtr1']
text = ('Nonpetroleum goods and services imports '+
        f'{c_line(imcol)} were equivalent to '+
        f'{data.IM.iloc[-1]:.1f} percent of GDP '+
        f'in {date}, while exports of nonpetroleum '+
        f'goods and services {c_line(excol)} were '+
        f'equivalent to {data.EX.iloc[-1]:.1f} percent '+
        f'of GDP. In {pc_date}, nonpetroleum imports '+
        f'were {data.IM.loc[pc_dt]:.1f} percent of GDP, '+
        f'and exports were {data.EX.loc[pc_dt]:.1f} percent.')
write_txt(text_dir / 'exim.txt', text)
print(text)

Nonpetroleum goods and services imports (see {\color{blue!90!cyan}\textbf{---}}) were equivalent to 12.9 percent of GDP in the third quarter of 2023, while exports of nonpetroleum goods and services (see {\color{green!60!teal!80!black}\textbf{---}}) were equivalent to 10.0 percent of GDP. In 2019 Q4, nonpetroleum imports were 13.0 percent of GDP, and exports were 10.6 percent.


### Goods Import Penetration

In [30]:
s = ['A353RC']
G = nipa_df(retrieve_table('T10205')['Data'], s).sort_index()

s = ['A253RC', 'A255RC', 'B647RC', 'LA000004', 'A650RC', 
     'B651RC', 'A652RC', 'A653RC', 'B648RC']
MX1 = nipa_df(retrieve_table('T40205A')['Data'], s).sort_index()
MX2 = nipa_df(retrieve_table('T40205B')['Data'], s).sort_index()
MX = pd.concat([MX1, MX2])

D = G['A353RC'] - MX['A253RC'] + MX['A255RC']
result = (MX['A255RC'] / D)

import_categories = ['B647RC', 'LA000004', 'A650RC', 'B651RC', 
                     'A652RC', 'A653RC', 'B648RC']
Msh = MX[import_categories].div(MX['A255RC'], axis=0)

Msh['Consumer'] = Msh['B647RC'] + Msh['A652RC'] + Msh['B651RC']
Msh['Capital'] = Msh['LA000004'] - Msh['B648RC'] + Msh['A650RC'] + Msh['A653RC']

final = Msh[['Consumer', 'Capital', 'B648RC']].multiply(result, axis=0) * 100

final.loc['1989':].to_csv(data_dir / 'goodsimpsh.csv', index_label='date')

In [31]:
final = pd.read_csv(data_dir / 'goodsimpsh.csv', index_col='date', 
                    parse_dates=True)
ltdate = final.index[-1]
ltdt = dtxt(ltdate)['qtr1']
cons, capi = [value_text(final[i].iloc[-1], style='eq') 
              for i in ['Consumer', 'Capital']]
oil = final['B648RC'].iloc[-1]
text = (f'In {ltdt}, the US imported nonpetroleum consumer goods '+
        f'{cons} of domestic consumption of '+
        'goods (see\cbox{cyan!40!white}). Petroleum-related imports claim '+
        f'{oil:.1f} percent (see\cbox{{purple}}), and imports of all other '+
        f'goods, primarily capital goods, industrial supplies, and materials, '+
        f'are {capi} (see\cbox{{blue!50!cyan}}).')
write_txt(text_dir / 'goodsimpsh1.txt', text)
print(text, '\n')

ch11 = (final.loc['2011-01-01'] - final.iloc[0])
cons11, pet11, oth11 = [value_text(ch11[i], adj='equivalent', threshold=0.1) 
                        for i in ['Consumer', 'B648RC', 'Capital']]
chlt = (final.iloc[-1] - final.loc['2011-01-01'])
conslt, petlt, othlt = [value_text(chlt[i], adj='equivalent', threshold=0.1) 
                        for i in ['Consumer', 'B648RC', 'Capital']]
text = ('From 1989 to 2011, imports of consumer goods excluding '+
        f'petroleum {cons11} of domestic consumption of goods; petroleum-'+
        f'related imports {pet11}; and all other goods imports {oth11}. \n\n'+
        f'Since 2011, imports of consumer goods {conslt} of domestic goods '+
        f'demand; imports of petroleum products {petlt}; and other '+
        f'imports {othlt}.')
write_txt(text_dir / 'goodsimpsh.txt', text)
print(text)

In 2023 Q3, the US imported nonpetroleum consumer goods equivalent to 14.8 percent of domestic consumption of goods (see\cbox{cyan!40!white}). Petroleum-related imports claim 2.7 percent (see\cbox{purple}), and imports of all other goods, primarily capital goods, industrial supplies, and materials, are equivalent to 14.3 percent (see\cbox{blue!50!cyan}). 

From 1989 to 2011, imports of consumer goods excluding petroleum increased by the equivalent of six percent of domestic consumption of goods; petroleum-related imports increased by the equivalent of 6.1 percent; and all other goods imports increased by the equivalent of 6.1 percent. 

Since 2011, imports of consumer goods decreased by the equivalent of 1.6 percent of domestic goods demand; imports of petroleum products decreased by the equivalent of 5.7 percent; and other imports decreased by the equivalent of 1.8 percent.


In [31]:
#nipa_series_codes(retrieve_table('T10502'))

### GDP Composition

In [32]:
# Contribution to growth from expenditure approach categories
s = ['DPCERY', 'A006RY', 'A822RY', 'A019RY', 'A191RL']
df = nipa_df(retrieve_table('T10502')['Data'], s).loc['1989':]
df.to_csv(data_dir / 'comp.csv', index_label='date')

# Text for chart on expenditure approach contributions
ltdt = dtxt(df.index[-1])['qtr2']
sl = [('DPCERY', 'contribution_to'), ('A006RY', 'contribution_to'), 
      ('A822RY', 'contribution'), ('A019RY', 'contribution')]
d = {s: value_text(df[s].iloc[-1], style=style, ptype='pp', 
                   digits=2, threshold=0.1) for s, style in sl}
text = (f'In {ltdt}, consumer spending (see\cbox{{yellow!80!orange}}) '+
        f'{d["DPCERY"]} overall real GDP growth. Private domestic '+
        f'investment (see\cbox{{blue!70!black}}) {d["A006RY"]} real GDP '+
        'growth, government spending and investment (see\cbox{cyan!50!white}) '+
        f'{d["A822RY"]}, and net exports (see\cbox{{green!60!black}}) '+
        f'{d["A019RY"]}.')  
write_txt(text_dir / 'gdp_exp_basic.txt', text)
print(text)

In the third quarter of 2023, consumer spending (see\cbox{yellow!80!orange}) contributed 2.69 percentage points to overall real GDP growth. Private domestic investment (see\cbox{blue!70!black}) contributed 1.47 percentage points to real GDP growth, government spending and investment (see\cbox{cyan!50!white}) contributed 0.79 percentage point, and net exports (see\cbox{green!60!black}) did not contribute.


### Contribution to change since 2019

In [33]:
# expenditure approach data
ex_s = ['A191RX', 'DPCERX', 'A006RX', 'A019RX', 'A822RX']
ex_df = nipa_df(retrieve_table('T10106')['Data'], ex_s).loc['1989':]
ex_v = {'A191RX': ['\\textbf{Real GDP} \\\\ \\textit{(annual rate)}', 
                   'red!95!black', 'white', 4.2], 
        'DPCERX': ['Consumer Spending', 'yellow!80!orange', 'black', 3],
        'A006RX': ['Private\\\\ Investment', 'blue!70!black', 'white', 2],
        'A822RX': ['Gov Spending \& Investment', 'cyan!50!white', 'black', 1],
        'A019RX': ['Net Exports', 'green!60!black', 'white', 0]}


# income approach data
inc_s = ['A261RC', 'A4002C', 'W271RC', 'indirect', 'A262RC']
s = ['A261RX', 'W256RX']
rgdi = nipa_df(retrieve_table('T11706')['Data'], s).dropna()

s = ['A261RC', 'A4002C', 'W056RC', 'A107RC', 'W271RC', 'A262RC',
     'A4102C', 'A038RC']

df = nipa_df(retrieve_table('T11000')['Data'], s).dropna()

# Calculate indirect taxes net of transfers
df['indirect'] = df['W056RC'] - df['A107RC']

# Calculate GDI deflator from real GDI series
deflator = rgdi['A261RX'] / df['A261RC']
deflator = deflator / deflator.iloc[-1]
inc_df = df.multiply(deflator, axis=0)
inc_v = {'A261RC': ['\\textbf{Real GDI} \\\\ \\textit{(annual rate)}', 
                    'blue!95!black', 'white', 4.2], 
         'A4002C': ['Labor', 'magenta!90!blue', 'white', 3],
         'W271RC': ['Capital', 'yellow!60!orange', 'black', 2],
         'indirect': ['Indirect Taxes Less Subsidies', 'violet', 'white', 1],
         'A262RC': ['Depreciation', 'teal!60!white', 'black', 0]}

# houshold inputs data
hh_s = ['GDP_idx', 'POP_idx', 'EPOP_sa_idx', 'AAH_trend_idx', 'LPROD_idx']
hh_df = pd.read_csv(data_dir / 'gdpjobslvl.csv', index_col='date', 
                 parse_dates=True)[hh_s].dropna()
hh_v = {'GDP_idx': ['\\textbf{Real GDP} \\\\ \\textit{(annual rate)}', 
                    'red!95!black', 'white', 4.2], 
        'POP_idx': ['Population', 'lime!90!green', 'black', 2],
        'EPOP_sa_idx': ['Employment Rate', 'green!30!teal!80!black', 'white', 1],
        'AAH_trend_idx': ['Average \\\\ Workweek', 'blue', 'white', 0],
        'LPROD_idx': ['Labor \\\\ Productivity', 'cyan!55!white', 'black', 3]}

# Charts for each approach
charts = {'exp': [ex_s, ex_df, ex_v], 'inc': [inc_s, inc_df, inc_v],
          'hhinp': [hh_s, hh_df, hh_v]}
for n, [s, df, v] in charts.items():
    d19 = df.loc['2019-10-01':]
    contrib = (((((d19.iloc[-1] - d19.iloc[0])
                  / d19[s[0]].iloc[0]) + 1)
                ** (1/((len(d19)-1)/4))) - 1) * 100
    # Settings for plot
    cmax = contrib.max()
    cmin = contrib.min()
    crng = cmax - cmin
    cbuf = max([(cmax - 0), (0 - cmin)]) * 0.68 #Buffer for text labels
    thresh = crng * 0.21 #Bigger bars labeled inside
    empty_neg = f'\\addplot[white!0] coordinates {{(-{cbuf:.2f}, 0)}};'
    empty_pos = f'\\addplot[white!0] coordinates {{({cbuf:.2f}, 0)}};'
    txt = [empty_neg, empty_pos]

    for k, [name, color, tcolor, y] in v.items():
        x = contrib[k].round(3)
        bar = f'\\addplot[{color}] coordinates {{({x}, {y})}};'
        vtc = 'black'
        tx = f'{contrib[k]:.1f}'
        if abs(x) > thresh:  # Some value labels inside of bars
            vt = f'\scriptsize \color{{{tcolor}}} \\textbf{{{tx}}}'
            inside = True
        else:
            vt = f'\scriptsize {tx}'
            inside = False
        if x > 0:
            ytlab = 'left, align=right'
            vtlab = 'left, align=right' if inside == True else 'right, align=right'
        else:
            ytlab = 'right, align=left'
            vtlab = 'right, align=left' if inside == True else 'left, align=left'
        # Create ylabel and value label
        ylabel = f'\\node[{ytlab}, text width=2.0cm] at (axis cs:0,{y}) {{{name}}};'
        vlabel = f'\\node[{vtlab}] at (axis cs:{x},{y}) {{{vt}}};'
        txt.append(bar)
        txt.append(ylabel)
        txt.append(vlabel)
    nodes = '\n'.join(txt)
    write_txt(text_dir / f'gdp_{n}_ch19.txt', nodes)

    # Store latest date
    write_txt(text_dir / f'gdp_{n}_ch19_dt.txt', dtxt(d19.index[-1])['qtr1'])

### Gross Domestic Income

In [34]:
s = ['A261RX', 'W256RX']
rgdi = nipa_df(retrieve_table('T11706')['Data'], s).dropna()

s = ['A261RC', 'A4002C', 'W056RC', 'A107RC', 'W271RC', 'A262RC',
     'A4102C', 'A038RC']

df = nipa_df(retrieve_table('T11000')['Data'], s).dropna()

pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']

# Calculate indirect taxes net of transfers
df['indirect'] = df['W056RC'] - df['A107RC']

# Calculate net domestic income (GDI less depreciation)
df['NDI'] = df['A261RC'] - df['A262RC']

# Calculate GDI deflator from real GDI series
deflator = rgdi['A261RX'] / df['A261RC']
deflator = deflator / deflator.iloc[-1]
df = df.multiply(deflator, axis=0)

# Calculate contributions to growth
dft = df.diff()
dft = dft.div(dft['A261RC'], axis=0)
contr = dft.multiply((((df['A261RC'].pct_change() + 1) ** 4) - 1) * 100, axis=0)

cols = ['A261RC', 'W271RC', 'A4002C', 'A262RC', 'indirect']

contr.loc['1989':, cols].to_csv(data_dir / 'gdi.csv', index_label='date')

dfpop = df.div(pop, axis=0).dropna()

ltdt = dtxt(dfpop.index[-1])['qtr1']
prdt = dtxt(dfpop.index[-2])['qtr1']
write_txt(text_dir / 'gdi_ltdt.txt', ltdt)

dfpop[cols].to_csv(data_dir / 'gdi_pc_comp.csv', index_label='date')

In [35]:
srs = {'A261RC': '\hspace{0.1mm} Gross Domestic Income', 
       'A4002C': '\hspace{-0.2mm} {\color{magenta!90!blue}\\textbf{---}} Labor', 
       'A4102C': '\hspace{4mm} Wages and Salaries',
       'A038RC': '\hspace{4mm} Supplements',
       'W271RC': '\hspace{-0.2mm} {\color{yellow!60!orange}\\textbf{---}} Profit', 
       'indirect': '\hspace{-0.1mm} {\color{violet}\\textbf{---}} Indirect Taxes', 
       'W056RC': '\hspace{4mm} Taxes on Production \& Imports',
       'A107RC': '\hspace{4mm} Less: Subsidies',
       'A262RC': '\hspace{-0.2mm} {\color{teal!60!white}\\textbf{---}} Depreciation'}

res = (dfpop[list(srs.keys())] * 1000).dropna()
lt = res.rename(srs, axis=1).iloc[-1]
lt.name = dtxt(lt.name)['qtr1']
pr = res.rename(srs, axis=1).iloc[-2]
pr.name = dtxt(pr.name)['qtr1']
pc = res.rename(srs, axis=1).loc['2019-10-01']
pc.name = dtxt(pc.name)['qtr1']
p12 = res.rename(srs, axis=1).loc['2012-01-01']
p12.name = dtxt(p12.name)['qtr1']
p00 = res.rename(srs, axis=1).loc['2000-01-01']
p00.name = dtxt(p00.name)['qtr1']
fi = res.rename(srs, axis=1).loc['1989-01-01']
fi.name = dtxt(fi.name)['qtr1']
table = pd.concat([lt, pr, pc, p12, p00, fi], axis=1).applymap('{:,.0f}'.format)
table.iloc[0, 0] = f'\${table.iloc[0, 0]}'
table.to_csv(data_dir / 'gdipc_levels.tex', sep='&', 
             lineterminator='\\\ ', quotechar=' ')

# GDI text
ltval = df['A261RC'].iloc[-1] / 1000
ltpc = dfpop['A261RC'].iloc[-1] * 1000
text = (f'of \${ltval:,.0f} billion in {ltdt}, equivalent to '+
        f'\${ltpc:,.0f} per capita. ')
write_txt(text_dir / 'gdi_levels.txt', text)
print('\n', text, '\n')

# NDI text
ltval = df['NDI'].iloc[-1] / 1000
ltpc = dfpop['NDI'].iloc[-1] * 1000
text = (f'is \${ltval:,.0f} billion in {ltdt}, or '+
        f'\${ltpc:,.0f} per capita. ')
write_txt(text_dir / 'ndi_levels.txt', text)
print('\n', text, '\n')

# Labor and capital share
lsh = (df['A4002C'] / df['NDI']) * 100
csh = (df['W271RC'] / df['NDI']) * 100
dsh = (df['A262RC'] / df['A261RC']) * 100

# Text by income type
prdate = '2019-10-01'
l_pc = dfpop['A4002C'].iloc[-1] * 1000
l_pr = dfpop['A4002C'].loc[prdate] * 1000
k_pc = dfpop['W271RC'].iloc[-1] * 1000
k_pr = dfpop['W271RC'].loc[prdate] * 1000
t_pc = dfpop['indirect'].iloc[-1] * 1000
t_pr = dfpop['indirect'].loc[prdate] * 1000
d_pc = dfpop['A262RC'].iloc[-1] * 1000
d_pr = dfpop['A262RC'].loc[prdate] * 1000

text = (f'Labor receives {lsh.iloc[-1]:.1f} percent of NDI in {ltdt}. '+
        'Gross labor income per capita is equivalent '+
        f'to \${l_pc:,.0f} in {ltdt} '+
        '(see {\color{magenta!90!blue}\\textbf{---}}) '+
        f'and \${l_pr:,.0f} in 2019 Q4, on an annualized, '+
        'seasonally-adjusted, and inflation-adjusted basis.\n\n '+
        f'Profits comprise {csh.iloc[-1]:.1f} percent of NDI in {ltdt}. '+
        f'Profits per person total \${k_pc:,.0f} in {ltdt} '+
        '(see {\color{yellow!60!orange}\\textbf{---}}) and '+
        f'\${k_pr:,.0f} in {prdt}, following the same adjustments. '+
        f'Indirect taxes less subsidies per capita total \${t_pc:,.0f} '+
        f'in {ltdt} (see {{\color{{violet}}\\textbf{{---}}}}) '+
        f'and \${t_pr:,.0f} in {prdt}.\n\nLastly, depreciation per '+
        f'capita is \${d_pc:,.0f} in {ltdt} (see '+
        '{\color{teal!60!white}\\textbf{---}}) and '+
        f'\${d_pr:,.0f} in {prdt}. Depreciation makes up '+
        f'{dsh.iloc[-1]:.1f} percent of GDI in {ltdt}. ')
write_txt(text_dir / 'gdi_levels_pc.txt', text)
print(text)


 of \$26,643 billion in 2023 Q2, equivalent to \$79,528 per capita.  


 is \$22,086 billion in 2023 Q2, or \$65,926 per capita.  

Labor receives 64.2 percent of NDI in 2023 Q2. Gross labor income per capita is equivalent to \$42,354 in 2023 Q2 (see {\color{magenta!90!blue}\textbf{---}}) and \$40,899 in 2019 Q4, on an annualized, seasonally-adjusted, and inflation-adjusted basis.

 Profits comprise 28.0 percent of NDI in 2023 Q2. Profits per person total \$18,433 in 2023 Q2 (see {\color{yellow!60!orange}\textbf{---}}) and \$18,175 in 2023 Q1, following the same adjustments. Indirect taxes less subsidies per capita total \$5,139 in 2023 Q2 (see {\color{violet}\textbf{---}}) and \$5,174 in 2023 Q1.

Lastly, depreciation per capita is \$13,602 in 2023 Q2 (see {\color{teal!60!white}\textbf{---}}) and \$12,418 in 2023 Q1. Depreciation makes up 17.1 percent of GDI in 2023 Q2. 


In [36]:
s = ['A261RC', 'A4002C', 'W271RC', 'indirect', 'A262RC']
df = contr[s]
gdi_lt = value_text(df["A261RC"].iloc[-1], adj='annual')
gdi_pr = value_text(df["A261RC"].iloc[-2], style='increase_of')
gdi_pr2 = value_text(df["A261RC"].iloc[-3], style='increase_of')

ltdt = dtxt(df.index[-1])['qtr2']
ltdt2 = dtxt(df.index[-1])['qtr1']
prdt = dtxt(df.index[-2])['qtr1']
prdt2 = dtxt(df.index[-3])['qtr1']

text1 = (f'In {ltdt}, gross domestic income {gdi_lt}, following {gdi_pr} '+
         f'in {prdt} and {gdi_pr2} in {prdt2}. ')  

l_lt = value_text(df["A4002C"].iloc[-1], style='contribution_to', 
                  digits=2, ptype='pp')
l_pr = value_text(df["A4002C"].iloc[-2], style='contribution_of', 
                  digits=2, ptype='pp')
k_lt = value_text(df["W271RC"].iloc[-1], style='contribution', 
                  digits=2, ptype='pp')
k_pr = value_text(df["W271RC"].iloc[-2], style='contribution', 
                  digits=2, ptype='pp')
t_lt = value_text(df['indirect'].iloc[-1], style='contribution_to', 
                  digits=2, ptype='pp')
t_pr = value_text(df['indirect'].iloc[-2], style='contribution', 
                  digits=2, ptype='pp')
    
text = (f'{text1}In the latest quarter, labor income {l_lt} overall growth, '+
        f'following {l_pr} in {prdt}. Profit income {k_lt} in '+
        f'{ltdt} and {k_pr} in {prdt}. Changes in indirect tax revenue '+
        f'and surpluses {t_lt} aggregate income growth in the latest quarter and '+
        f'{t_pr} in {prdt}. ')
write_txt(text_dir / 'gdi_growth_comp.txt', text)
print(text) 

In the second quarter of 2023, gross domestic income increased at an annual rate of 0.7 percent, following an increase of 0.5 percent in 2023 Q1 and a decrease of three percent in 2022 Q4. In the latest quarter, labor income contributed 2.25 percentage points to overall growth, following a contribution of 2.37 percentage points in 2023 Q1. Profit income subtracted 1.77 percentage points in the second quarter of 2023 and subtracted 2.42 percentage points in 2023 Q1. Changes in indirect tax revenue and surpluses subtracted 0.23 percentage point from aggregate income growth in the latest quarter and contributed 0.03 percentage point in 2023 Q1. 


### Consumer Spending Overview (Levels)

In [34]:
pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']

n = {'DPCERC': 'Total',
     'DGDSRC': 'Goods',
     'DSERRC': 'Services',
     'DHUTRC': 'Housing',
     'A011RC': 'ResInv',
     'DMOTRC': 'MotorVeh',
     'DFDHRC': 'Furn',
     'DREQRC': 'RecDG',
     'DFXARC': 'Groc',
     'DCLORC': 'Cloth',
     'DHLCRC': 'Health',
     'DTRSRC': 'Transp',
     'DRCARC': 'RecSer',
     'DFSARC': 'FoodAcc',
     'DIFSRC': 'FinIns'}
n2 = {k[:-2] + 'RA': v for k, v in n.items()}
s = list(n.keys())
s2 = list(n2.keys())

othserv = lambda x: x['Services'] - x['Housing']
shelter = lambda x: x['Housing'] + x['ResInv']

df = (nipa_df(retrieve_table('T10505')['Data'], s)
      .rename(n, axis=1))
df2 = (nipa_df(retrieve_table('T10503')['Data'], s2)
       .rename(n2, axis=1))
real = ((df2 / df2.iloc[-1]) * df.iloc[-1]).assign(OTHSERV = othserv, SHELTER = shelter)
pp = real.divide(pop, axis=0)

keep_col = ['Goods', 'Services', 'Housing', 'ResInv', 'OTHSERV', 'SHELTER']
pp.loc['1989':, keep_col].to_csv(data_dir / 'pce_levels.csv', index_label='date')

lttot = real.Total.iloc[-1] / 1_000_000
prtot = real.Total.iloc[-2] / 1_000_000
pctot = real.Total.loc['2019-10-01'] / 1_000_000
ltdate = dtxt(real.index[-1])['qtr1']
prdate = dtxt(real.index[-2])['qtr1']
totpp = pp.Total.iloc[-1] * 1_000
goodpp = pp.Goods.iloc[-1] * 1_000
servpp = pp.Services.iloc[-1] * 1_000
goodpppc = pp.Goods.loc['2019-10-01'] * 1_000
servpppc = pp.Services.loc['2019-10-01'] * 1_000

text = ('Total consumer spending is '+
        f'\${lttot:.1f} trillion in {ltdate}, compared to a price-adjusted '+
        f'\${prtot:.1f} trillion in {prdate} and \${pctot:.1f} trillion in 2019 Q4. '+
        'On a per person basis, consumer spending is '+
        f'\${totpp:,.0f} in {ltdate}, of which \${goodpp:,.0f} are spent on goods '+
        '(see {\color{red}\\textbf{---}}) and '+
        f'\${servpp:,.0f} on services '+
        '(see {\color{orange}\\textbf{---}}). In the fourth quarter of 2019, '+
        f'before the pandemic, consumer spending on goods was \${goodpppc:,.0f} '+
        f'per person, and spending on services was \${servpppc:,.0f} per person, '+
        'after adjusting for inflation. ')
write_txt(text_dir / 'pce_levels.txt', text)
print(text)

hult = pp['Housing'].iloc[-1] * 1_000
hupc = pp['Housing'].loc['2019-10-01'] * 1_000
rfilt = pp['ResInv'].iloc[-1] * 1_000
rfipc = pp['ResInv'].loc['2019-10-01'] * 1_000

text = ('Within consumer spending on services, housing and utilities spending '+
        f'totals \${hult:,.0f} on an annualized and per person basis in {ltdate} '+
        '(see {\color{green!60!blue}\\textbf{---}}) '+
        f'and \${hupc:,.0f} in 2019 Q4. Construction or improvement '+
        'of housing is considered residential fixed investment, not '+
        'consumer spending, but can be combined with spending to analyze '+
        'patterns in shelter costs. In '+
        f'{ltdate}, residential investment totals \${rfilt:,.0f} per person '+
        '(see {\color{blue!80!black}\\textbf{---}}), '+
        f'compared to \${rfipc:,.0f} in the pre-COVID data covering 2019 Q4. ')
write_txt(text_dir / 'pce2_levels.txt', text)
print('\n', text)

othlt = pp['OTHSERV'].iloc[-1] * 1_000
othpr = pp['OTHSERV'].iloc[-2] * 1_000
othpc = pp.loc['2019-10-01', 'OTHSERV'] * 1_000
chval = ((othlt / othpc) - 1) * 100
chtxt = value_text(chval)
shellt = pp['SHELTER'].iloc[-1] * 1_000
shelpr = pp['SHELTER'].iloc[-2] * 1_000
shelpc = pp.loc['2019-10-01', 'SHELTER'] * 1_000
shelmax = pp['SHELTER'].max() * 1_000
shelmaxdt = dtxt(pp.SHELTER.idxmax())['qtr2']

text = ('Consumer spending on services other than housing and utilities totals \$'+
        f'{othlt:,.0f} per person, on an annualized basis, in {ltdate} '+
        '(see {\color{blue!75!white}\\textbf{---}}), '+
        f'compared to \${othpr:,.0f} in {prdate}, and \${othpc:,.0f} in '+
        f'2019 Q4. Spending on non-housing services has {chtxt} since 2019 Q4. '+
        '\n\nShelter costs, which combine housing, utilities, and residential fixed '+
        f'investment, are \${shellt:,.0f} per person in {ltdate} '+
        '(see {\color{green!85!blue}\\textbf{---}})'+
        f', \${shelpr:,.0f} '+
        f'in {prdate}, and \${shelpc:,.0f} in 2019 Q4. Shelter spending peaked at '+
        f'\${shelmax:,.0f} per person in {shelmaxdt}, during the housing bubble.')
write_txt(text_dir / 'pce3_levels.txt', text)
print('\n', text)

Total consumer spending is \$18.7 trillion in 2023 Q3, compared to a price-adjusted \$18.6 trillion in 2023 Q2 and \$17.0 trillion in 2019 Q4. On a per person basis, consumer spending is \$55,840 in 2023 Q3, of which \$18,569 are spent on goods (see {\color{red}\textbf{---}}) and \$37,270 on services (see {\color{orange}\textbf{---}}). In the fourth quarter of 2019, before the pandemic, consumer spending on goods was \$15,909 per person, and spending on services was \$35,542 per person, after adjusting for inflation. 

 Within consumer spending on services, housing and utilities spending totals \$9,877 on an annualized and per person basis in 2023 Q3 (see {\color{green!60!blue}\textbf{---}}) and \$9,405 in 2019 Q4. Construction or improvement of housing is considered residential fixed investment, not consumer spending, but can be combined with spending to analyze patterns in shelter costs. In 2023 Q3, residential investment totals \$3,208 per person (see {\color{blue!80!black}\textbf{-

In [35]:
n = {'Total': 'Consumer Spending',
     'Goods': '\hspace*{-0.6mm} {\color{red}\\textbf{---}} Goods',
     'MotorVeh': '\hspace{4mm} Motor Vehicles \& Parts',
     'Furn': '\hspace{4mm} Furniture \& HH Equipment',
     'RecDG': '\hspace{4mm} Recreational Durable Goods',
     'Groc': '\hspace{4mm} Groceries',
     'Cloth': '\hspace{4mm} Clothes \& Shoes',
     'OTHSERV': '\hspace*{-0.6mm} {\color{blue!75!white}\\textbf{---}} Services Excluding Shelter',
     'Health': '\hspace{4mm} Health Care Services',
     'Transp': '\hspace{4mm} Transportation',
     'RecSer': '\hspace{4mm} Recreational',
     'FoodAcc': '\hspace{4mm} Food \& Accommodations',
     'FinIns': '\hspace{4mm} Financial \& Insurance',
     'SHELTER': '\hspace*{-0.6mm} {\color{green!85!blue}\\textbf{---}} Shelter Including Investment',
     'Housing': '\hspace{4mm} Housing Services \& Utilities ',
     'ResInv': '\hspace{4mm} Residential Fixed Investment'}

res = pp[n.keys()].rename(n, axis=1) * 1_000
lt = res.iloc[-1]
lt.name = dtxt(lt.name)['qtr1']
pr = res.iloc[-2]
pr.name = dtxt(pr.name)['qtr1']
py = res.iloc[-5]
py.name = dtxt(py.name)['qtr1']
pc = res.loc['2019-10-01']
pc.name = dtxt(pc.name)['qtr1']
p00 = res.loc['2000-01-01']
p00.name = dtxt(p00.name)['qtr1']
fi = res.loc['1989-01-01']
fi.name = dtxt(fi.name)['qtr1']
table = pd.concat([lt, pr, py, pc, p00, fi], axis=1).applymap('{:,.0f}'.format)
table.columns = table.columns.str.replace(' ', '\n')
table.iloc[0, 0] = f'\${table.iloc[0, 0]}'
table.to_csv(data_dir / 'pce_levels.tex', sep='&', lineterminator='\\\ ', quotechar=' ')
#table

### Consumer Spending and Residential Fixed Investment

In [36]:
s = ['DNPIRY', 'DSERRY', 'DPCERY', 'DGDSRY', 'DMOTRY',
     'DFDHRY', 'DREQRY', 'DFXARY', 'DCLORY', 'DHLCRY',
     'DTRSRY', 'DRCARY', 'DFSARY', 'DIFSRY', 'DHUTRY',
     'A011RY']

n = {'TOTAL': '& Consumer Spending',
     'DGDSRY': '\cbox{red} & Goods',
     'DMOTRY': '& \hspace{1mm} Motor Vehicles \& Parts',
     'DFDHRY': '& \hspace{1mm} Furniture \& HH Equipment',
     'DREQRY': '& \hspace{1mm} Recreational Durable Goods',
     'DFXARY': '& \hspace{1mm} Groceries',
     'DCLORY': '& \hspace{1mm} Clothes \& Shoes',
     'OTHSERV': '\cbox{blue!75!white} & Services Excluding Shelter',
     'DHLCRY': '& \hspace{1mm} Health Care Services',
     'DTRSRY': '& \hspace{1mm} Transportation',
     'DRCARY': '& \hspace{1mm} Recreational',
     'DFSARY': '& \hspace{1mm} Food \& Accommodations',
     'DIFSRY': '& \hspace{1mm} Financial \& Insurance',
     'SHELTER': '\cbox{green!85!blue} & Shelter Including Investment',
     'DHUTRY': '& \hspace{1mm} Housing Services \& Utilities ',
     'A011RY': '& \hspace{1mm} Residential Fixed Investment'}

total = lambda x: x['DPCERY']
othserv = lambda x: x['DSERRY'] - x['DHUTRY']
shelter = lambda x: x['DHUTRY'] + x['A011RY']

df = (nipa_df(retrieve_table('T10502')['Data'], s)
      .assign(TOTAL = total, OTHSERV = othserv, SHELTER = shelter)
      [list(n.keys())])

(df.loc['1988':, ['DGDSRY', 'OTHSERV', 'SHELTER']]
 .rolling(4).mean().dropna()
 .to_csv(data_dir / 'pce_ma.csv', index_label='date', float_format='%g'))

data = pd.concat([df.iloc[-3:].iloc[::-1].T, df.iloc[-5]], axis=1)
cols = [f'& {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data.columns)]
data.columns = cols
data['1-year'] = df.rolling(4).mean().iloc[-1]
data['10-year'] = df.rolling(40).mean().iloc[-1]
data['30-year'] = df.rolling(120).mean().iloc[-1]
data.index = data.index.map(n)
(data.round(2).applymap('{:,.2f}'.format)
 .to_csv(data_dir / 'pce.tex', sep='&', lineterminator='\\\ ', quotechar=' '))

ld = dtxt(df.index[-1])['qtr1']
prd = dtxt(df.index[-2])['qtr1']
ld2 = dtxt(df.index[-1])['qtr2']

totlt = df['TOTAL'].iloc[-1]
totltt = value_text(totlt, style='contribution_to', ptype='pp', 
                    digits=1, threshold=0.1)
totpr = df['TOTAL'].iloc[-2]
totprt = value_text(totpr, style='contribution', ptype='pp', 
                    digits=1, threshold=0.1)
totpc = df.loc['2019-10-01', 'TOTAL']
totpct = value_text(totpc, style='contribution_of', ptype='pp', 
                    digits=1, casual=True, threshold=0.1)

txt1 = (f'These categories {totltt} GDP growth in {ld} and {totprt} in '+
        f'{prd}, compared to {totpct} in 2019 Q4, before the pandemic.')
write_txt(text_dir / 'pce1.txt', txt1)
print(txt1)

gdslt = df['DGDSRY'].iloc[-1]
gdsltt = value_text(gdslt, style='contribution_to', ptype='pp', 
                    digits=1, threshold=0.1)
serlt = df['OTHSERV'].iloc[-1]
serltt = value_text(serlt, style='contribution', ptype='pp', digits=1, 
                    casual=True, threshold=0.1)
shelt = df['SHELTER'].iloc[-1]
sheltt = value_text(shelt, style='contribution', ptype='pp', 
                    digits=1, threshold=0.1)

txt2 = (f'In {ld2}, household spending on goods {gdsltt} GDP growth, '+
         'household spending on services other than housing and utilities '+
        f'{serltt}, and shelter spending and investment {sheltt}.')
write_txt(text_dir / 'pce2.txt', txt2)
print('\n', txt2)

These categories contributed 2.7 percentage points to GDP growth in 2023 Q3 and contributed 0.6 percentage point in 2023 Q2, compared to an addition of 1.7 percentage points in 2019 Q4, before the pandemic.

 In the third quarter of 2023, household spending on goods contributed 1.1 percentage points to GDP growth, household spending on services other than housing and utilities added 1.2 percentage points, and shelter spending and investment contributed 0.6 percentage point.


### Trade Contribution to GDP

In [37]:
s = ['A019RY', 'A253RY', 'A646RY', 'A255RY', 'A656RY']
df = nipa_df(retrieve_table('T10502')['Data'], s)
df.loc['1989':].to_csv(data_dir / 'nx.csv', index_label='date')

sl = [('A253RY', 'contribution_to'), ('A646RY', 'contribution'), 
      ('A255RY', 'contribution_to'), ('A656RY', 'contribution')]

d = {}
for s, style in sl:
    d[s] = value_text(df[s].iloc[-1], style, 'pp', digits=2)
    
ldate = dtxt(df.index[-1])['qtr2']

text = (f"Goods exports {d['A253RY']} GDP growth in {ldate} while "+
        f"services exports {d['A646RY']}. Good imports {d['A255RY']} "+
        f"GDP growth and services imports {d['A656RY']}.")
write_txt(text_dir / 'trade.txt', text)
print(text)

Goods exports contributed 0.54 percentage point to GDP growth in the third quarter of 2023 while services exports contributed 0.14 percentage point. Good imports subtracted 0.63 percentage point from GDP growth and services imports subtracted 0.12 percentage point.


### Business investment

In [38]:
# Contributions to growth
s = ['Y001RY', 'A009RY', 'Y033RY']
df = nipa_df(retrieve_table('T10502')['Data'], s)
df.loc['1989':].to_csv(data_dir / 'businv.csv', index_label='date')

# By category/type
s1 = ['Y001RC', 'B009RC', 'Y033RC']
s = s1 + ['A008RC']
data = nipa_df(retrieve_table('T10105')['Data'], s)
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
sh = data.divide(gdp, axis=0) * 100
sh.loc['1989':].to_csv(data_dir / 'businvsh.csv', index_label='date')

# End nodes
colors = {'Y001RC': 'violet', 'B009RC': 'yellow!50!orange',
          'Y033RC': 'cyan!60!white'}
adj = node_adj(sh[s])
date = {n: None for n, c in colors.items()}
imax = sh[s1].iloc[-1].idxmax()
adj[imax] += 0.32
date[imax] = 'q'
nodes = '\n'.join([end_node(sh[n], c, date=date[n], offset=adj[n])
                   for n, c in colors.items()])
write_txt(text_dir / f'bus_inv_sh_nodes.txt', nodes)

# Text
cl = {n: c_line(c) for n, c in colors.items()}
ltdt = dtxt(df1.index[-1])['qtr1']
levels, shgdp = {}, {}
for series in s1:
    level_val = data[series].iloc[-1] / 1_000
    levels[series] = f"\${level_val:,.0f} billion"
    shgdp[series] = f"{sh[series].iloc[-1]:.1f} percent of GDP"
text = ('Business gross investment in fixed assets is grouped into '+
        'three categories: structures, equipment, and intellectual '+
        'property (for example software and R\&D). Annualized '+
        f'investment in structures is {levels["B009RC"]} in {ltdt}, '+
        f'equivalent to {shgdp["B009RC"]} {cl["B009RC"]}. Equipment '+
        f'investment is {levels["Y033RC"]} or {shgdp["Y033RC"]} '+
        f'{cl["Y033RC"]}, and intellectual property investment '+
        f'is {levels["Y001RC"]} or {shgdp["Y001RC"]} {cl["Y001RC"]}.')
write_txt(text_dir / 'businv_sh.txt', text)
print(text)

Business gross investment in fixed assets is grouped into three categories: structures, equipment, and intellectual property (for example software and R\&D). Annualized investment in structures is \$832 billion in 1998 Q4, equivalent to 3.0 percent of GDP (see {\color{yellow!50!orange}\textbf{---}}). Equipment investment is \$1,384 billion or 5.0 percent of GDP (see {\color{cyan!60!white}\textbf{---}}), and intellectual property investment is \$1,502 billion or 5.4 percent of GDP (see {\color{violet}\textbf{---}}).


In [39]:
# Gross vs Net
s = ['W790RC', 'W276RC', 'W987RC']
df = nipa_df(retrieve_table('T50100')['Data'], s).join(data['A008RC'])
df['NFI'] = (df['A008RC'] - df['W276RC']) # Net Fixed Investment
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
res = (df.div(gdp, axis=0) * 100).dropna()
res.loc['1989':].to_csv(data_dir / 'businv_main.csv', index_label='date')

ltdt1 = dtxt(df.index[-1])['qtr1']
ltdt2 = dtxt(df.index[-1])['qtr2']
shgdp, levels = {}, {}
for series in ['W790RC', 'W276RC', 'W987RC', 'A008RC', 'NFI']:
    level_val = df[series].iloc[-1] / 1_000
    n = ''
    if level_val < 0:
        n = 'negative '
        level_val = abs(level_val)
    levels[series] = f"{n}\${level_val:,.0f} billion"
    shgdp[series] = f"{res[series].iloc[-1]:.1f} percent of GDP"
    
diff = cagr(df.loc['2019-10-01':, 'W987RC'])
gdt = value_text(diff, adj='annual')
diff2 = cagr(df.loc['2019-10-01':, 'NFI'])
if diff2 < -95:
    ndt = 'but collapsed completely'
else:
    ndt = 'and ' + value_text(diff2, adj='annual')
gpcl = f"{n}\${df.loc['2019-10-01', 'W987RC'] / 1_000:,.0f} billion"
npcl = f"{n}\${df.loc['2019-10-01', 'NFI'] / 1_000:,.0f} billion"
if diff < 0:
    txt = 'as gross investment outpaced depreciation'
else:
    txt = 'as growth of depreciation costs outpaced the increase in gross investment'
    
text = (f'In {ltdt2}, gross private business investment totals '+
        f'{levels["W987RC"]} on a seasonally-adjusted annualized basis, '+
        f'equivalent to {shgdp["W987RC"]} '+
        '(see {\color{blue!60!violet}\\textbf{---}}). Private business investment '+
        f'in fixed assets totals {levels["A008RC"]}, or {shgdp["A008RC"]} '+
        '(see {\color{cyan!80!white}\\textbf{---}}). Private business depreciation '+
        f'totals {levels["W276RC"]} in the quarter, or {shgdp["W276RC"]} '+
        '(see {\color{magenta}\\textbf{---}}). '+
        f'As a result, net fixed investment is {levels["NFI"]}, or {shgdp["NFI"]} '+
        '(see {\color{green!80!blue}\\textbf{---}}).\n\n '+
        'In 2019 Q4, prior to the COVID-19 pandemic, private business gross '+
        f'investment was {gpcl}. Since 2019 Q4, gross investment {gdt}. '+
        f'Net fixed investment was {npcl} in 2019 Q4, {ndt} from 2019 Q4 to {ltdt1}, '+
        f'{txt}. ')
write_txt(text_dir / 'businv_levels.txt', text)
print(text)  

In the third quarter of 2023, gross private business investment totals \$3,709 billion on a seasonally-adjusted annualized basis, equivalent to 13.4 percent of GDP (see {\color{blue!60!violet}\textbf{---}}). Private business investment in fixed assets totals \$3,718 billion, or 13.5 percent of GDP (see {\color{cyan!80!white}\textbf{---}}). Private business depreciation totals \$2,992 billion in the quarter, or 10.8 percent of GDP (see {\color{magenta}\textbf{---}}). As a result, net fixed investment is \$726 billion, or 2.6 percent of GDP (see {\color{green!80!blue}\textbf{---}}).

 In 2019 Q4, prior to the COVID-19 pandemic, private business gross investment was \$2,935 billion. Since 2019 Q4, gross investment increased at an annual rate of 6.4 percent. Net fixed investment was \$711 billion in 2019 Q4, and increased at an annual rate of 0.6 percent from 2019 Q4 to 2023 Q3, as growth of depreciation costs outpaced the increase in gross investment. 


### Business Investment Table and Text

In [40]:
# Contribution to growth by business investment type
s = ['A008RY', 'A009RY', 'Y033RY', 'Y034RY', 'B935RY', 
     'A680RY', 'A681RY', 'Y001RY', 'B985RY', 'Y006RY']

n = { 'A008RY': 'Total',
      'A009RY': '\hspace{-2mm}\cbox{yellow!50!orange}Structures',
      'Y033RY': '\hspace{-2mm}\cbox{cyan!60!white}Equipment',
      'Y034RY': '\hspace{4mm} Information Processing',
      'B935RY': '\hspace{6mm} Computers \& Peripherals',
      'A680RY': '\hspace{4mm} Industrial Equipment',
      'A681RY': '\hspace{4mm} Transportation Equipment',
      'Y001RY': '\hspace{-2mm}\cbox{violet}Intellectual Property Products',
      'B985RY': '\hspace{4mm} Software',
      'Y006RY': '\hspace{4mm} Research \& Development'}

df = (nipa_df(retrieve_table('T10502')['Data'], s)
      [list(n.keys())])
data = df.iloc[-3:].iloc[::-1].T
data[df.index[-5]] = df.iloc[-5]
data[df.index[-9]] = df.iloc[-9]
data.columns = [f' {q.year} Q{q.quarter}' 
                if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
                for i, q in enumerate(data.columns)]
data['1-year'] = df.rolling(4).mean().iloc[-1].round(2)
data['10-year'] = df.rolling(40).mean().iloc[-1].round(2)
data['30-year'] = df.rolling(120).mean().iloc[-1].round(2)
data.index = data.index.map(n)
res = data.applymap('{:,.2f}'.format)
res.to_csv(data_dir / 'businv.tex', sep='&', 
           lineterminator='\\\ ', quotechar=' ')

In [41]:
# Text
sl = [('A008RY', 'contribution_to'), ('A009RY', 'contribution_to'), 
      ('Y033RY', 'contribution'), ('Y001RY', 'contribution'), 
      ('A680RY', 'contribution')]
d = {}
for s, style in sl:
    value = df[s].iloc[-1]
    d[s] = (value_text(df[s].iloc[-1], style,
           ptype='pp', digits=2, threshold=0.02))

t_1y = value_text(df['A008RY'].iloc[-4:].mean(), 'contribution_of', adj='average',
           ptype='pp', digits=2, threshold=0.02).replace('an', 'the')
ld = dtxt(df.index[-1])['qtr1']
ld2 = dtxt(df.index[-1])['qtr2']
val1y = df.A008RY.rolling(4).mean().iloc[-1]
vallt = df.A008RY.iloc[-1]
compare = compare_text(vallt, val1y, [0.1, 0.5, 2.0])
colors = {'Y001RY': 'violet', 'A009RY': 'yellow!50!orange',
          'Y033RY': 'cyan!60!white'}
cb = {n: c_box(c) for n, c in colors.items()}

text = (f"Private business gross fixed investment {d['A008RY']} "+
        f"annualized GDP growth in {ld}, {compare} {t_1y} "+
        f"over the past year. In {ld}, investment in structures "+
        f"{d['A009RY']} GDP growth {cb['A009RY']}, investment in "+
        f"equipment {d['Y033RY']} {cb['Y033RY']}, and investment "+
        f"in intellectual property products {d['Y001RY']} {cb['Y001RY']}.")
write_txt(text_dir / 'businv.txt', text)
print(text)

Private business gross fixed investment did not contribute to annualized GDP growth in 2023 Q3, slightly below the average contribution of 0.49 percentage point over the past year. In 2023 Q3, investment in structures contributed 0.05 percentage point to GDP growth (see\cbox{yellow!50!orange}), investment in equipment subtracted 0.19 percentage point (see\cbox{cyan!60!white}), and investment in intellectual property products contributed 0.14 percentage point (see\cbox{violet}).


### Durable goods new orders

In [42]:
# New orders for capital goods excluding defense or aircraft
url = ('https://api.census.gov/data/timeseries/eits/advm3?'+
       f'get=cell_value,time_slot_id&key={census_key}&'+
       'category_code=NXA&data_type_code=NO&time=from+1992&'+
       'for=us&seasonally_adj=yes')
r = requests.get(url).json()
date = lambda x: pd.to_datetime(x.time)
df = (pd.DataFrame(r[1:], columns=r[0]).assign(date = date)
        .set_index('date')['cell_value'].astype('float')).sort_index()
df.to_csv(data_dir / 'dgno_raw.csv', index_label='date', 
          header=True)

In [43]:
df = pd.read_csv(data_dir / 'dgno_raw.csv', index_col='date', 
                 parse_dates=True)['cell_value']
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])
res = ((df.resample('QS').sum() * 4  / 
        gdp['A191RC']).dropna() * 100).iloc[1:]
(res.rename('value').to_csv(data_dir / 'dgno.csv', 
                            index_label='date',  header=True))
color = 'purple!50!violet'
node = end_node(res, color, date='q')
write_txt(text_dir / 'dgno_node.txt', node)
ltval = f'\${df.iloc[-1] / 1000:,.0f} billion'
ldate = dtxt(df.index[-1])['mon1']
comp = pd.to_datetime('2020-02-01')
compdt = dtxt(pd.to_datetime(comp))['mon1']
val = value_text(df.pct_change(12).iloc[-1] * 100)
pcv = ((df.iloc[-1] / df.loc[comp]) - 1) * 100
pcval = value_text(pcv, 'increase_by')
text = ('New orders for manufactured core capital goods excluding '+
        f'aircraft total {ltval} in {ldate}, equivalent to '+
        f'{res.iloc[-1]:.1f} percent of GDP {c_line(color)}. New '+
        f'orders {val} over the past year, and {pcval} since '+
        f'{compdt}.')
write_txt(text_dir / 'dgno.txt', text)
print(text)

New orders for manufactured core capital goods excluding aircraft total \$74 billion in September 2023, equivalent to 3.2 percent of GDP (see {\color{purple!50!violet}\textbf{---}}). New orders increased 2.4 percent over the past year, and increased by 21.8 percent since February 2020.


### Government spending and investment

In [44]:
n = {'A822RY': 'Consolidated Government Total',
     'A823RY': '\hspace{2mm}Federal Total',
     'A824RY': '\cbox{blue!60!black}National Defense',
     'A997RY': '\hspace{6mm}Consumption Expenditures',
     'A788RY': '\hspace{6mm}Gross Investment',
     'A825RY': '\cbox{green!85!black}Federal Non-Defense',
     'A542RY': '\hspace{6mm}Consumption Expenditures',
     'A798RY': '\hspace{6mm}Gross Investment',
     'A829RY': '\hspace{-2mm}\cbox{purple!70!magenta}State \& Local Total',
     'A991RY': '\hspace{4mm}Consumption Expenditures',
     'A799RY': '\hspace{4mm}Gross Investment'}

df = nipa_df(retrieve_table('T10502')['Data'], 
             list(n.keys()) + ['A191RL'])
df.rolling(4).mean().loc['1989':].to_csv(data_dir / 'gov.csv', 
                                         index_label='date')

In [45]:
ltdt = dtxt(df.index[-1])['qtr2']
ltdt2 = dtxt(df.index[-1])['qtr1']
tot = df.A822RY
gdp = df['A191RL'].iloc[-1]
oneyr = value_text(tot.iloc[-4:].mean(), 'contribution', 'pp', 
                   threshold=0.01, digits=2)
v89 = tot.loc['1989':].mean()
totval = value_text(tot.iloc[-1], 'contribution_to', 'pp', 
                    threshold=0.01, digits=2)
defval = value_text(df['A824RY'].iloc[-4:].mean(), 'contribution', 'pp', 
                   threshold=0.01, digits=2)
defcb = c_box('blue!60!black')
fedval = value_text(df['A825RY'].iloc[-4:].mean(), 'contribution', 'pp', 
                   threshold=0.01, digits=2)
fedcb = c_box('green!85!black')
slgval = value_text(df['A829RY'].iloc[-4:].mean(), 'contribution', 'pp', 
                   threshold=0.01, digits=2)
slgcb = c_box('purple!70!magenta')
text = ('Government consumption and investment directly affect economic '+
        f'growth in the short-term. In {ltdt}, government consumption spending '+
        f'and investment {totval} the real GDP growth rate of {gdp:.1f} '+
        'percent. Over the last four quarters, government consumption '+
        f'and investment {oneyr} to economic growth, on average. '+
        f'Since 1989, the average contribution has been {v89:.2f} '+
        f'percentage points.\n\nOver the four quarters ending {ltdt2}, '+
        f'by level of government, national defense {defval} {defcb}, '+
        f'federal non-defense {fedval} {fedcb}, and state and '+
        f'local government {slgval} {slgcb}.')
write_txt(text_dir / 'gov.txt', text)
print(text)

# Table 
res = df[n.keys()]
data = res.iloc[-3:].iloc[::-1].T
data[res.index[-5]] = res.iloc[-5]
data[res.index[-9]] = res.iloc[-9]
cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data.columns)]

data.columns = cols
data['1-year'] = res.rolling(4).mean().iloc[-1].round(2)
data['10-year'] = res.rolling(40).mean().iloc[-1].round(2)
data['30-year'] = res.rolling(120).mean().iloc[-1].round(2)
data.index = data.index.map(n)
data = data.applymap('{:.2f}'.format)
data.to_csv(data_dir / 'gov.tex', sep='&', lineterminator='\\\ ', quotechar=' ')

Government consumption and investment directly affect economic growth in the short-term. In the third quarter of 2023, government consumption spending and investment contributed 0.79 percentage point to the real GDP growth rate of 4.9 percent. Over the last four quarters, government consumption and investment contributed 0.77 percentage point to economic growth, on average. Since 1989, the average contribution has been 0.25 percentage points.

Over the four quarters ending 2023 Q3, by level of government, national defense contributed 0.18 percentage point (see\cbox{blue!60!black}), federal non-defense contributed 0.17 percentage point (see\cbox{green!85!black}), and state and local government contributed 0.43 percentage point (see\cbox{purple!70!magenta}).


In [3]:
#nipa_series_codes(retrieve_table('T30100'))

### Government Summary

In [4]:
# GDP
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

# Government data Table 3.1
d = {'W021RC': 'Current receipts',
     'W054RC': 'Current tax receipts',
     'W782RC': 'Contributions for government social insurance',
     'W022RC': 'Current expenditures',
     'A955RC': 'Consumption Expenditures',
     'A084RC': 'Current transfer payments',
     'A180RC': 'Interest Payments',
     'A107RC': 'Subsidies'}
df = nipa_df(retrieve_table('T30100')['Data'], d.keys()).rename(d, axis=1)
df['Taxes'] = (df[['Current tax receipts', 
                   'Contributions for government social insurance']]
              ).sum(axis=1)
df['Other'] = (df['Current receipts'] - df['Taxes'])
df['Transfers and Subsidies'] = (df[['Current transfer payments', 'Subsidies']]
                                ).sum(axis=1)

res = df.divide(gdp, axis=0).dropna() * 100
ex = pd.concat([res.iloc[i] for i in [-1, -17]], axis=1).T.reset_index()
ex.to_csv(data_dir / 'gov_overview.csv', index_label='date')

# Text
ltdt = dtxt(ex['index'][0])['qtr1']
prdt = dtxt(ex['index'][1])['qtr1']
ltrec = value_text(ex['Current receipts'][0], 'plain')
prrec = value_text(ex['Current receipts'][1], 'plain')
ltexp = value_text(ex['Current expenditures'][0], 'plain')
prexp = value_text(ex['Current expenditures'][1], 'plain')

text = ("As an overview of the consolidated government's effect "+
        'on GDP, government '+
        f'receipts are equivalent to {ltrec} of GDP in {ltdt}, '+
        f'compared to {prrec} in {prdt}. The vast majority of '+
        'these receipts are taxes, including social insurance '+
        'contributions.\n\n Government spending is equivalent to '+
        f'{ltexp} of GDP in the latest data and {prexp} '+
        'in 2019. This includes consumption expenditures, which are '+
        "the government's operating costs, transfers and subsidies, "+
        'and interest payments. These are covered in more detail in '+
        'the following subsections.')
write_txt(text_dir / 'gov_summary.txt', text)
print(text)

As an overview of the consolidated government's effect on GDP, government receipts are equivalent to 27.0 percent of GDP in 2023 Q2, compared to 27.6 percent in 2019 Q2. The vast majority of these receipts are taxes, including social insurance contributions.

 Government spending is equivalent to 33.8 percent of GDP in the latest data and 33.2 percent in 2019. This includes consumption expenditures, which are the government's operating costs, transfers and subsidies, and interest payments. These are covered in more detail in the following subsections.


In [5]:
# Nodes and labels
s1 = (['Taxes'], 'Current receipts', 
      {'Taxes': ('blue!50!cyan', 'Taxes'), 
       'Other': ('violet', 'Other')})
s2 = (['Interest Payments', 'Transfers and Subsidies', 
      'Consumption Expenditures'], 'Current expenditures',
      {'Interest Payments': ('orange!90!red', 'Interest'), 
       'Transfers and Subsidies': 
       ('gray', 'Transfers \& Subsidies'),
       'Consumption Expenditures': ('blue!60!gray', 'Operations')})

for s in [s1, s2]:
    node = []
    n = s[1][8:11]
    for i in [0, 1]:
        # Date node
        dn = (f"\\node[below, align=center] at (axis cs: {i},0) {{"+
              f"\\scriptsize \color{{black!80}} {dtxt(ex['index'][i])['qtr3']} "+
              "\\\\[-0.22em] \\scriptsize \color{black!80} "+
              f"{dtxt(ex['index'][i])['year']}}};")
        node.append(dn)
        cs = ex[s[2].keys()].cumsum(axis=1) - (ex[s[2].keys()] / 2)
        for c in s[2].keys():
            col = 'black!90' if c == 'Transfers and Subsidies' else 'white'
            v = ex[c][i]
            hv = cs[c][i]
            tv = f'{v:.1f}'
            # Value label node
            lnode = (f'\\node[align=center] at (axis cs: {i},{hv}) '+
                    f'{{\scriptsize \color{{{col}}} {tv}}};')
            if c != 'Other':
                node.append(lnode)
            if i == 0:
                # Name label node
                nnode = (f'\\node[left, align=right, xshift=-9pt, text width=42pt] '+
                         f'at (axis cs: 0,{hv}) '+
                        f'{{\\footnotesize \color{{{s[2][c][0]}}} {s[2][c][1]}}};')
                node.append(nnode)
        tnv = ex[s[1]][i]
        tnvt = f'{tnv:.1f}'
        # Top label node (sum)
        tn = (f'\\node[above, align=center] at (axis cs: {i},{tnv}) '+
              f'{{\scriptsize \color{{black!90}} {tnvt}}};')
        node.append(tn)
    nodes = '\n'.join(node)
    write_txt(text_dir / f'gov_summary_{n}_nodes.txt', nodes)

### Government Net Investment

In [48]:
cofc = nipa_df(retrieve_table('T11000')['Data'], ['A264RC'])['A264RC']
ginv = nipa_df(retrieve_table('T30100')['Data'], ['A782RC'])['A782RC']
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

data = (((ginv - cofc) / gdp) * 100).loc['1989':]
data.name = 'Value'
data.to_csv(data_dir / 'govnetinv.csv', index_label='date')

color = 'red'
node = end_node(data, color, date='q', digits=2, offset=0.35)
write_txt(text_dir / 'govnetinv_node.txt', node)

ltdt = dtxt(data.index[-1])['qtr1']
yrdt = dtxt(data.index[-5])['qtr1']
yr2dt = dtxt(data.index[-9])['qtr1']
gni = f'\${(ginv - cofc).iloc[-1] / 1_000:.1f} billion'
gnish = f'{data.iloc[-1]:.2f} percent'
gnishyr = f'{data.iloc[-5]:.2f} percent'
gnishyr2 = f'{data.iloc[-9]:.2f} percent'

text = ("Government gross investment, less depreciation, is the government's "+
        "net investment in the tangible assets that make the economy more "+
        "productive. Government investment includes infrastructure, "+
        "buildings, equipment, intellectual property, and other capital goods. "+
        f"In the latest data, covering {ltdt}, government net investment is "+
        f"{gni}. Government net investment is equivalent to {gnish} "+
        f"of GDP in {ltdt} {c_line(color)}, compared "+
        f"to {gnishyr} in {yrdt}, and {gnishyr2} in {yr2dt}. ")
write_txt(text_dir / f'govnetinv.txt', text)
print(text)

Government gross investment, less depreciation, is the government's net investment in the tangible assets that make the economy more productive. Government investment includes infrastructure, buildings, equipment, intellectual property, and other capital goods. In the latest data, covering 2023 Q3, government net investment is \$230.7 billion. Government net investment is equivalent to 0.84 percent of GDP in 2023 Q3 (see {\color{red}\textbf{---}}), compared to 0.58 percent in 2022 Q3, and 0.67 percent in 2021 Q3. 


### Government receipts and expenditures

In [3]:
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

d = {'slggdp': {'name': 'combined state and local government', 
                'table': 'T30300', 'series': 
                {'W024RC': 'SLG_EXP', 'W023RC': 'SLG_REC'}},
     'fedgdp': {'name': 'federal government', 
                'table': 'T30200', 'series':
                {'W005RC': 'FED_REC', 'W013RC': 'FED_EXP'}}}

data = pd.DataFrame()

ltdate = dtxt(gdp.index[-1])['qtr1']

for group, details in d.items():
    df = nipa_df(retrieve_table(details['table'])['Data'], details['series'].keys())
    (df.div(gdp, axis=0) * 100).loc['1989':].to_csv(data_dir / f'{group}.csv', 
                                                    index_label='date', float_format='%g')
    
    grp = group[:3].upper()
    df.columns = [details['series'][i] for i in df.columns]
    bal = df[f'{grp}_REC'] - df[f'{grp}_EXP']
    bal_lt = bal.dropna().iloc[-1]
    bal_lt_dt = dtxt(bal.dropna().index[-1])['qtr1']
    def_sur = 'deficit' if bal_lt < 0 else 'surplus'
    df[f'{grp}_BAL'] = bal
    
    for col in df.columns:
        data[col] = df[col]
        data[col+'_GDP'] = df[col].div(gdp, axis=0) * 100
        
    bal_gdp = data[f'{grp}_BAL_GDP'].dropna().iloc[-1]
    bal_txt = (f'In {bal_lt_dt}, the {d[group]["name"]} {def_sur} was '+
               f'\${abs(bal_lt) / 1000:,.0f} billion or {abs(bal_gdp):.1f} percent of GDP. ')
    
    exp_txt = (f'{d[group]["name"].capitalize()} expenditures total '+
               f'\${data[grp+"_EXP"].iloc[-1] / 1000000:.1f} trillion, '+
               f'or {data[grp+"_EXP_GDP"].iloc[-1]:.1f} percent of GDP, in {ltdate}. ')
    
    if pd.isna(df[[i for i in df.columns if i[4:] == 'REC'][0]].iloc[-1]) == True:
        rec_txt = (f'BEA has not yet released receipts data for {ltdate}, however, '+
                   f'in {bal_lt_dt}, {d[group]["name"]} receipts total '+
                   f'\${data[grp+"_REC"].dropna().iloc[-1] / 1000000:.1f} trillion, '+
                   f'or {data[grp+"_REC_GDP"].dropna().iloc[-1]:.1f} percent of GDP. ')
    else:
        rec_txt = ('Receipts for the same period total '+
                   f'\${data[grp+"_REC"].dropna().iloc[-1] / 1000000:.1f} trillion '+
                   f'or {data[grp+"_REC_GDP"].dropna().iloc[-1]:.1f} percent of GDP. ')
    
    text = exp_txt + rec_txt + bal_txt
    write_txt(text_dir / f'{group}.txt', text)
    print(text)

Combined state and local government expenditures total \$3.8 trillion, or 13.7 percent of GDP, in 2023 Q3. BEA has not yet released receipts data for 2023 Q3, however, in 2023 Q2, combined state and local government receipts total \$3.6 trillion, or 13.2 percent of GDP. In 2023 Q2, the combined state and local government deficit was \$185 billion or 0.7 percent of GDP. 
Federal government expenditures total \$6.4 trillion, or 23.1 percent of GDP, in 2023 Q3. BEA has not yet released receipts data for 2023 Q3, however, in 2023 Q2, federal government receipts total \$4.7 trillion, or 17.3 percent of GDP. In 2023 Q2, the federal government deficit was \$1,661 billion or 6.1 percent of GDP. 


In [5]:
d = {'SLG_EXP_GDP': '\hspace{3mm} Expenditures ',
     'SLG_REC_GDP': '\hspace{3mm} Receipts ',
     'SLG_BAL_GDP': '\hspace{3mm} Surplus (+) / Deficit (-) ',
     'FED_EXP_GDP': '\hspace{3mm} Expenditures',
     'FED_REC_GDP': '\hspace{3mm} Receipts',
     'FED_BAL_GDP': '\hspace{3mm} Surplus (+) / Deficit (-)'}

cats = {'FED': ' \ \\textbf{Federal Government}', 
        'SLG': ' \ \\textbf{State \& Local Government}'}
final = pd.DataFrame()
for cat, name in cats.items():
    cols = [i for i in data.columns 
            if i.endswith('GDP') & i.startswith(cat)]
    tmp = data[cols]
    table = pd.concat([tmp.iloc[-3:].iloc[::-1].T, 
                       tmp.iloc[-5].T, 
                       tmp.iloc[-9].T, 
                       tmp.iloc[-17].T], axis=1)
    table.columns = [dtxt(i)['qtr4'] for i in table.columns]
    for i in [16, 40, 120]:
        table[f'{i/4:.0f}-year'] = tmp.iloc[-i:].mean()
    table = table.applymap('{:.1f}'.format).sort_index(ascending=False)
    blank = pd.Series(index=table.columns, dtype=float)
    blank['index'] = name
    table = table.reset_index()
    table.index = table.index + 1
    table.loc[0] = blank
    table = table.sort_index().set_index('index').rename(d)
    table.index.name = ''
    final = pd.concat([final, table]).replace('nan', '--')
final.to_csv(data_dir / 'gov_rec_exp_bal.tex', sep='&', 
             lineterminator='\\\ ', quotechar=' ')

In [51]:
#data['FED_BAL_GDP'].iloc[-13:].plot(kind='bar')

### Federal Government Current Tax Receipts

In [6]:
# Federal 
s = {'W006RC': 'Current Tax Receipts', 
     'A074RC': 'Personal Current Taxes',
     'B075RC': 'Taxes on Corporate Income', 
     'W780RC': 'Social Insurance Contributions'}
other = lambda x: x.W006RC - x.A074RC - x.B075RC
data = nipa_df(retrieve_table('T30200')['Data'], 
               s.keys()).assign(Other = other)

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
sh = (data.divide(gdp, axis=0) * 100).dropna()
sh.loc['1989':].to_csv(data_dir / 'fed_gov_rec_type.csv', 
                       index_label='date')

# end nodes
srs = {'A074RC': 'green!50!black', 'B075RC': 'purple!80!violet',
       'W780RC': 'blue!60!white', 'Other': 'black!30!white'}

# date on max value
mon = {k: None for k, v in srs.items()}
mon[sh[srs.keys()].iloc[-1].idxmax()] = 'q'
       
nodes = '\n'.join([end_node(sh[n], col, date=mon[n]) 
                   for n, col in srs.items()])
write_txt(text_dir / 'fed_gov_rec_type_nodes.txt', nodes)

# Text overview
ltdt = dtxt(data.index[-1])['qtr1']
taxsrs = ['W006RC', 'W780RC']
taxval = data[taxsrs].iloc[-1].sum() / 1_000_000
taxsh = value_text(sh[taxsrs].iloc[-1].sum(), 'plain')
text = ('Taxes and social insurance contributions are the main '+
        f'federal government receipts, and total \${taxval:.1f} '+
        f'trillion in {ltdt}, equivalent to {taxsh} of GDP.')
write_txt(text_dir / 'fed_gov_rec_overview.txt', text)
print(text, '\n\n')

cl = {n: c_line(color) for n, color in srs.items()}
pctsh = value_text(sh['A074RC'].iloc[-1], 'plain')
pctlg = value_text(sh['A074RC'].mean(), 'plain')
sish = value_text(sh['W780RC'].iloc[-1], 'plain')
silg = value_text(sh['W780RC'].mean(), 'plain')
ci19 = value_text(sh.loc['2019', 'B075RC'].mean(), 'plain')
cish = value_text(sh['B075RC'].iloc[-1], 'plain')
othsh = value_text(sh['Other'].iloc[-1], 'plain')
# Text categories
text = (f'As of {ltdt}, federal personal current tax receipts '+
        f'are equivalent of {pctsh} of GDP {cl["A074RC"]}. '+
        'Some volatility in these receipts comes from swings '+
        'in yearly capital gains. Since 1989, these tax receipts '+
        f'average {pctlg} of GDP. \n\nSocial insurance '+
        'contributions are relatively stable over time and comprise '+
        f'{sish} of GDP in {ltdt} {cl["W780RC"]}, compared '+
        f'to an average of {silg} since 1989.\n\n'+
        'Corporate income tax receipts are typically two percent of '+
        'GDP during economic expansions, but were cut to '+
        f'{ci19} in 2019. In {ltdt}, these receipts '+
        f'are equivalent to {cish} of GDP {cl["B075RC"]}.\n\n'+
        'Other tax receipts, including excise taxes and customs duties '+
        f'total {othsh} of GDP in the latest data {cl["Other"]}.')
write_txt(text_dir / 'fed_gov_rec_cats.txt', text)
print(text)

Taxes and social insurance contributions are the main federal government receipts, and total \$1.8 trillion in 2023 Q3, equivalent to 16.9 percent of GDP. 


As of 2023 Q3, federal personal current tax receipts are equivalent of eight percent of GDP (see {\color{green!50!black}\textbf{---}}). Some volatility in these receipts comes from swings in yearly capital gains. Since 1989, these tax receipts average 7.9 percent of GDP. 

Social insurance contributions are relatively stable over time and comprise 6.6 percent of GDP in 2023 Q3 (see {\color{blue!60!white}\textbf{---}}), compared to an average of 6.6 percent since 1989.

Corporate income tax receipts are typically two percent of GDP during economic expansions, but were cut to one percent in 2019. In 2023 Q3, these receipts are equivalent to 1.5 percent of GDP (see {\color{purple!80!violet}\textbf{---}}).

Other tax receipts, including excise taxes and customs duties total 0.8 percent of GDP in the latest data (see {\color{black!30!w

### State & Local Government Current Tax Receipts

In [7]:
# SLG
s = {'W023RC': 'Current Receipts', 
     'B089RC': 'Federal Transfers',
     'W070RC': 'Current Tax Receipts', 
     'W071RC': 'Personal Current Taxes',
     'B245RC': 'Income Taxes',
     'B248RC': 'Sales Taxes', 
     'LA000355': 'Property Taxes'}
other = lambda x: x.W023RC - x.B089RC - x.W070RC
data = nipa_df(retrieve_table('T30300')['Data'], s.keys()).assign(Other = other)

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
sh = (data.divide(gdp, axis=0) * 100).dropna()
sh.loc['1989':].to_csv(data_dir / 'slg_gov_rec_type.csv', index_label='date')

# end nodes
srs = {'W070RC': 'blue!60!black', 'B089RC': 'orange!90!red',
       'Other': 'black!30!white'}

# date on max value
mon = {k: None for k, v in srs.items()}
mon[sh[srs.keys()].iloc[-1].idxmax()] = 'q'
       
nodes = '\n'.join([end_node(sh[n], col, date=mon[n]) 
                   for n, col in srs.items()])
write_txt(text_dir / 'slg_gov_rec_type_nodes.txt', nodes)

# Nodes for small plots
cols = {'B245RC': 'blue!90!black', 
        'B248RC': 'blue!70!cyan', 
        'LA000355': 'cyan!80!white'}

for n, col in cols.items():
    write_txt(text_dir / f'slg_{n}_node.txt', end_node(sh[n], col))

In [8]:
# Text
ltdt = dtxt(sh.index[-1])['qtr1']
pryr = dtxt(sh.index[-5])['qtr1']
ltval = value_text(sh.W070RC.iloc[-1], 'plain')
prval = value_text(sh.W070RC.iloc[-5], 'plain')
fedsh = value_text(sh.B089RC.iloc[-1], 'plain')
fedpy = value_text(sh.B089RC.iloc[-5], 'plain')
fed89 = value_text(sh.loc['1989', 'B089RC'].mean(), 'plain')
othsh = value_text(sh.Other.iloc[-1], 'plain')
ch19 = value_text(sh.W070RC.iloc[-1] - 
                  sh.loc['2019', 'W070RC'].mean(), adj='equivalent')
itch19 = value_text(sh.B245RC.iloc[-1] - 
                    sh.loc['2019', 'B245RC'].mean(), adj='equivalent')
cl = {n: c_line(color) for n, color in {**cols, **srs}.items()}
text = (f'In {ltdt}, combined state and local government tax receipts '+
        f'are equivalent to {ltval} of GDP {cl["W070RC"]}. One year prior, '+
        f'in {pryr}, these tax receipts total {prval} of GDP. '+ 
        f'Since 2019, these tax receipts {ch19} of GDP, and income tax '+
        f'receipts {itch19} of GDP.\n\n'+
        'Federal government transfers to state and local governments total '+
        f'{fedsh} of GDP in {ltdt} {cl["B089RC"]}, and {fedpy} one year prior. '+
        'These transfers peaked during COVID-19 relief efforts, but have been '+
        f'climbing over time, from {fed89} of GDP in 1989. '+
        f'Other receipts are equivalent to {othsh} of GDP in {ltdt} {cl["Other"]}. ')
write_txt(text_dir / 'slg_gov_rec_cats.txt', text)
print(text)

In 2023 Q2, combined state and local government tax receipts are equivalent to 8.6 percent of GDP (see {\color{blue!60!black}\textbf{---}}). One year prior, in 2022 Q2, these tax receipts total 9.4 percent of GDP. Since 2019, these tax receipts decreased by the equivalent of 0.4 percent of GDP, and income tax receipts decreased by the equivalent of 0.3 percent of GDP.

Federal government transfers to state and local governments total 3.6 percent of GDP in 2023 Q2 (see {\color{orange!90!red}\textbf{---}}), and 3.8 percent one year prior. These transfers peaked during COVID-19 relief efforts, but have been climbing over time, from 1.6 percent of GDP in 1989. Other receipts are equivalent to 1.1 percent of GDP in 2023 Q2 (see {\color{black!30!white}\textbf{---}}). 


### Government Receipts Overview

In [11]:
s = ['W021RC', 'W054RC', 'W782RC']
taxsi = lambda x: x.W054RC + x.W782RC
data = nipa_df(retrieve_table('T30100')['Data'], s).assign(TAXSI = taxsi).dropna()

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
sh = (data.divide(gdp, axis=0) * 100).dropna()
sh.loc['1989':].to_csv(data_dir / 'gov_rec_overview.csv', index_label='date')

# end nodes
srs = {'W021RC': 'cyan!65!white', 'TAXSI': 'blue!50!violet'}

# date on max value
maxkey = sh[srs.keys()].iloc[-1].idxmax()
mon = {k: None for k, v in srs.items()}
mon[maxkey] = 'q'
off = {k: 0 for k, v in srs.items()}
off[maxkey] = 0.35
       
nodes = '\n'.join([end_node(sh[n], col, date=mon[n], offset=off[n]) 
                   for n, col in srs.items()])
write_txt(text_dir / 'gov_rec_overview_nodes.txt', nodes)

# Text
ltdt = dtxt(sh.index[-1])['qtr1']
rec_tot = data.W021RC.iloc[-1] / 1_000_000
cl = {n: c_line(color) for n, color in srs.items()}
totsh = value_text(sh.W021RC.iloc[-1], 'plain')
taxsh = value_text(sh.TAXSI.iloc[-1], 'plain')
taxcomp = value_text((data.TAXSI.iloc[-1] / 
                      data.W021RC.iloc[-1]) * 100, 'plain')

text = (f'Government receipts total \${rec_tot:.1f} trillion in {ltdt}, '+
        f'equivalent to {totsh} of GDP {cl["W021RC"]}. Taxes and social '+
        f'insurance contributions comprise {taxcomp} of receipts and are '+
        f'equivalent to {taxsh} of GDP in {ltdt} {cl["TAXSI"]}. ')
write_txt(text_dir / 'gov_rec_overview.txt', text)
print(text)

Government receipts total \$7.3 trillion in 2023 Q2, equivalent to 27.0 percent of GDP (see {\color{cyan!65!white}\textbf{---}}). Taxes and social insurance contributions comprise 94.8 percent of receipts and are equivalent to 25.6 percent of GDP in 2023 Q2 (see {\color{blue!50!violet}\textbf{---}}). 


In [56]:
#nipa_series_codes(retrieve_table('T30300'))

### Government Spending Overview

In [55]:
srs = {'T30100': {'W022RC': 'Current Expenditures'},
       'T30200': {'W013RC': 'Federal Current Expenditures',
                  'B089RC': 'SLG Grants'},
       'T30300': {'W024RC': 'SLG Current Expenditures'}}

df = pd.concat([nipa_df(retrieve_table(t)['Data'], d.keys()).rename(d, axis=1) 
               for (t, d) in srs.items()], axis=1)
df['Federal'] = df['Federal Current Expenditures'] - df['SLG Grants']
df['SLG'] = df['SLG Current Expenditures'] - df['SLG Grants']
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
sh = (df.divide(gdp, axis=0) * 100)
sh.loc['1989':].to_csv(data_dir / 'gov_exp_overview.csv', 
                       index_label='date')

# End nodes
color = 'black!80!white'
node = end_node(sh['Current Expenditures'], color, date='q', offset=0.35)
write_txt(text_dir / 'gov_exp_tot_node.txt', node)

# Nodes for latest values
cols = ['Federal', 'SLG Grants', 'SLG']
sdf = sh[cols].iloc[-1]
height = ((sdf.cumsum() - (sdf / 2) + 2)).to_dict()
val = sdf.to_dict()
dtp = dtxt(sh.index[-1] + pd.DateOffset(months=2))['datetime']
nodes = [f'\\absnode{{{{{dtp}}}}}{{{height[i]}}}{{\scriptsize {val[i]:.1f}}}' 
         for i in cols]
nodetext = '\n'.join(nodes)
write_txt(text_dir / 'gov_exp_overview_nodes.txt', nodetext)

# Summary Text main
s = 'Current Expenditures'
ltdt = dtxt(sh.index[-1])['qtr1']
pydt = dtxt(sh.index[-5])['qtr1']
totval = df[s].iloc[-1] / 1_000_000
totsh = value_text(sh[s].iloc[-1], 'plain')
totshpy = value_text(sh[s].iloc[-5], 'plain')
totshcov = value_text(sh.loc['2020': '2021', s].mean(), 'plain')
totshlt = value_text(sh.loc['1989':, s].mean(), 'plain')

cl = c_line('black!80!white')

text = (f'As of {ltdt}, government current expenditures total \${totval:.1f} '+
        f'trillion, and are equivalent to {totsh} of GDP {cl}. One year prior, '+
        f'in {pydt}, government current expenditures were equivalent to {totshpy} '+
        'of GDP. Government spending peaked during the COVID-19 pandemic, averaging '+
        f'{totshcov} of GDP from 2020 to 2021. Since 1989, government spending '+
        f'averages {totshlt} of GDP.')
write_txt(text_dir / 'gov_exp_overview.txt', text)
print(text)

As of 2023 Q3, government current expenditures total \$9.3 trillion, and are equivalent to 33.5 percent of GDP (see {\color{black!80!white}\textbf{---}}). One year prior, in 2022 Q3, government current expenditures were equivalent to 33.6 percent of GDP. Government spending peaked during the COVID-19 pandemic, averaging 40.9 percent of GDP from 2020 to 2021. Since 1989, government spending averages 33.3 percent of GDP.


In [58]:
#nipa_series_codes(retrieve_table('T30100'))

In [56]:
# Table 
s = {'W022RC': f'Current Expenditures {c_line("black!80!white", see=False)}',
     'A955RC': '\hspace{2mm}Consumption Expenditures',
     'A084RC': '\hspace{2mm}Current Transfer Payments',
     'A180RC': '\hspace{2mm}Interest Payments',
     'A107RC': '\hspace{2mm}Subsidies'}
df = nipa_df(retrieve_table('T30100')['Data'], s)
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
res = (df.divide(gdp, axis=0) * 100)
data = res.iloc[-3:].iloc[::-1].T
data[res.index[-5]] = res.iloc[-5]
data[res.index[-9]] = res.iloc[-9]
cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data.columns)]

data.columns = cols
data['1-year'] = res.rolling(4).mean().iloc[-1]
data['10-year'] = res.rolling(40).mean().iloc[-1]
data['30-year'] = res.rolling(120).mean().iloc[-1]
data.index = data.index.map(s)
data = data.applymap('{:.1f}'.format)
data.to_csv(data_dir / 'gov_exp.tex', sep='&', lineterminator='\\\ ', quotechar=' ')

In [57]:
# text
ltdt = dtxt(res.index[-1])['qtr1']
pydt = dtxt(res.index[-5])['qtr1']
fedsh = value_text(sh.Federal.iloc[-1], 'plain')
fedpy = value_text(sh.Federal.iloc[-5], 'plain')
fedtr = value_text(sh['SLG Grants'].iloc[-1], 'plain')
slgsh = value_text(sh.SLG.iloc[-1], 'plain')
cbfed = c_box('blue!65!gray!90!white')
cbtr = c_box('cyan!40!white')
cbslg = c_box('red!20!purple!70!white')
consh = value_text(res.A955RC.iloc[-1], 'plain')
conpy = value_text(res.A955RC.iloc[-5], 'plain')
conlg = value_text(res.A955RC.iloc[-121:].mean(), 'plain')
trsh = value_text(res.A084RC.iloc[-1], 'plain')
trpy = value_text(res.A084RC.iloc[-5], 'plain')
trlg = value_text(res.A084RC.iloc[-121:].mean(), 'plain')
intsh = value_text(res.A180RC.iloc[-1], 'plain')
intpy = value_text(res.A180RC.iloc[-5], 'plain')
intlg = value_text(res.A180RC.iloc[-121:].mean(), 'plain')
subsh = value_text(res.A107RC.iloc[-1], 'plain')
subpy = value_text(res.A107RC.iloc[-5], 'plain')
text = ('By level of government, federal current expenditures are equivalent '+
        f'to {fedsh} of GDP in {ltdt}, and {fedpy} in '+
        f'{pydt} {cbfed}. Federal government transfers to state '+
        f'and local governments comprise {fedtr} of GDP in the '+
        f'latest data {cbtr}. State and local government current '+
        'expenditures, excluding transfers from the federal government, '+
        f'are equivalent to {slgsh} of GDP in {ltdt} {cbslg}. \n\nBy '+
        'category, consumption expenditures are equivalent to '+
        f'{consh} of GDP in {ltdt}, and {conpy} '+
        'of GDP one year prior. Over the past 30 years, consumption '+
        f'expenditures average {conlg} of GDP. Current '+
        'transfer payments, which are largely government social benefits, '+
        f'total {trsh} of GDP in {ltdt}, compared to '+
        f'{trpy} one year prior, and a long-term average '+
        f'of {trlg}.\n\nConsolidated government interest '+
        f'payments are equivalent to {intsh} of GDP in the '+
        f'latest data, compared to {intpy} one year prior. '+
        f'Interest payments comprise {intlg} over the past '+
        '30 years, on average. Government subsidies to businesses total '+
        f'{subsh} of GDP in {ltdt} and {subpy} in '+
        f'{pydt}. ')
write_txt(text_dir / 'gov_exp_overview2.txt', text)
print(text)

By level of government, federal current expenditures are equivalent to 19.8 percent of GDP in 2023 Q3, and 19.7 percent in 2022 Q3 (see\cbox{blue!65!gray!90!white}). Federal government transfers to state and local governments comprise 3.3 percent of GDP in the latest data (see\cbox{cyan!40!white}). State and local government current expenditures, excluding transfers from the federal government, are equivalent to 10.4 percent of GDP in 2023 Q3 (see\cbox{red!20!purple!70!white}). 

By category, consumption expenditures are equivalent to 13.7 percent of GDP in 2023 Q3, and 13.8 percent of GDP one year prior. Over the past 30 years, consumption expenditures average 14.8 percent of GDP. Current transfer payments, which are largely government social benefits, total 14.9 percent of GDP in 2023 Q3, compared to 15.5 percent one year prior, and a long-term average of 13.5 percent.

Consolidated government interest payments are equivalent to 4.6 percent of GDP in the latest data, compared to 3.8 

### Personal Income

See also table in generate_table notebook

In [58]:
# deflator
d = nipa_df(retrieve_table('T20304')['Data'], ['DPCERG'])['DPCERG']
deflator = d.iloc[-1] / d

# collect and combine series
s = ['A065RC', 'A033RC', 'A041RC', 'A048RC', 'W210RC', 'A577RC', 'A061RC']
df = (nipa_df(retrieve_table('T20100')['Data'], s)
      .assign(CAPITAL = lambda x: x['A041RC'] + x['A048RC'] + x['W210RC'],
              TRANSFER = lambda x: x['A577RC'] - x['A061RC'])
      .drop(['A061RC', 'A041RC', 'A048RC', 'W210RC', 'A577RC'], axis=1)
      .multiply(deflator, axis=0))
growth_contrib_ann(df, 'A065RC').loc['1989':].to_csv(data_dir / 'pi.csv', index_label='date')

data = growth_contrib_ann(df, 'A065RC').rename({'A065RC': 'TOTAL', 'A033RC': 'LABOR'}, axis=1)

ltdate = dtxt(data.index[-1])['qtr1']
tot = value_text(data['TOTAL'].iloc[-1], digits=2)
slist = ['LABOR', 'CAPITAL', 'TRANSFER']
style = {k: 'contribution' for k in slist}
style['LABOR'] = 'contribution_to'
d = {}
for i in slist:
    d[i] = value_text(data[i].iloc[-1], style=style[i], 
                      ptype='pp', digits=2)
text = (f'Aggregate real personal income {tot} over the year ending {ltdate}. '+
        f'Labor income {d["LABOR"]} overall growth, capital '+
        f'income {d["CAPITAL"]}, and net government benefits {d["TRANSFER"]}. ')
write_txt(text_dir / 'pi.txt', text)
print(text)

Aggregate real personal income increased 1.36 percent over the year ending 2023 Q3. Labor income contributed 1.25 percentage points to overall growth, capital income contributed 0.35 percentage point, and net government benefits subtracted 0.25 percentage point. 


### Government Personal Income

In [59]:
s = ['DPCERG']

d = nipa_df(retrieve_table('T20304')['Data'], s)['DPCERG']
deflator = d.iloc[-1] / d

s = ['B230RC']

population = nipa_df(retrieve_table('T20100')['Data'], s)['B230RC']

s = ['A063RC', 'B202RC', 'A061RC', 'W055RC']
df = nipa_df(retrieve_table('T20100')['Data'], s)

n = {'A063RC': 'Welfare',
     'B202RC': 'WandS'}
result = df.multiply(deflator, axis=0).divide(population, axis=0)
result['TaxSI'] = result['A061RC'] + result['W055RC']
result = result.drop(['A061RC', 'W055RC'], axis=1).rename(n, axis=1)
result.loc['1989':].to_csv(data_dir / 'govpi.csv', index_label='date')

ltdate = dtxt(result.index[-1])['qtr1']
prdt = '2019-10-01'
prdate = dtxt(pd.to_datetime(prdt))['qtr1']
wsltval = result['WandS'].iloc[-1] * 1_000
wsprval = result.loc[prdt, 'WandS'] * 1_000
wltval = result['Welfare'].iloc[-1] * 1_000
wprval = result.loc[prdt, 'Welfare'] * 1_000
w89val = result.loc['1989-01-01', 'Welfare'] * 1_000
tltval = result['TaxSI'].iloc[-1] * 1_000
tprval = result.loc[prdt, 'TaxSI'] * 1_000
t89val = result.loc['1989-01-01', 'TaxSI'] * 1_000

text = (f'In {ltdate}, government worker wages and salaries '+
        f'were equivalent to \${wsltval:,.0f} per capita, following a '+
        f'price-adjusted \${wsprval:,.0f} in {prdate} '+
        '(see {\color{orange}\\textbf{---}}). Net government benefits were equivalent '+
        f'to \${wltval:,.0f} per capita in {ltdate}, compared to \${wprval:,.0f} '+
        f'per capita in {prdate} '+
        '(see {\color{violet}\\textbf{---}}). In 1989 Q1, net benefits were '+
        f'equivalent to \${w89val:,.0f} per person.  \n\n Personal current taxes '+
        f'and social insurance contributions total \${tltval:,.0f} per capita '+
        f'in {ltdate}, \${tprval:,.0f} in {prdate}, and \${t89val:,.0f} in 1989 '+
        '(see {\color{cyan!80!blue}\\textbf{---}}).')
write_txt(text_dir / 'govpi.txt', text)
print(text)

In 2023 Q3, government worker wages and salaries were equivalent to \$5,203 per capita, following a price-adjusted \$5,199 in 2019 Q4 (see {\color{orange}\textbf{---}}). Net government benefits were equivalent to \$11,886 per capita in 2023 Q3, compared to \$10,940 per capita in 2019 Q4 (see {\color{violet}\textbf{---}}). In 1989 Q1, net benefits were equivalent to \$4,420 per person.  

 Personal current taxes and social insurance contributions total \$13,791 per capita in 2023 Q3, \$12,849 in 2019 Q4, and \$8,096 in 1989 (see {\color{cyan!80!blue}\textbf{---}}).


### Government Consumption and Investment

In [60]:
s = ['A823RC', 'A829RC', 'A191RC', 'A824RC', 'A825RC']
df = nipa_df(retrieve_table('T10105')['Data'], s).sort_index()
result = df.drop('A191RC', axis=1).divide(df['A191RC'], axis=0) * 100
result.loc['1989':].to_csv(data_dir / 'govci.csv', index_label='date')

ltdate = dtxt(result.index[-1])['qtr1']
prdt = pd.to_datetime('2019-10-01')
prdate = dtxt(prdt)['qtr1']
fndnom = df['A825RC'].iloc[-1] / 1_000
fndlt = result['A825RC'].iloc[-1]
fndpr = result.loc[prdt, 'A825RC']
dlt = result['A824RC'].iloc[-1]
dpr = result.loc[prdt, 'A824RC']
d89 = result.loc['1989-01-01', 'A824RC']
slt = result['A829RC'].iloc[-1]
spr = result.loc[prdt, 'A829RC']

text = (f'In {ltdate}, federal non-defense spending and investment '+
        f'was \${fndnom:,.1f} billion, equivalent to {fndlt:.1f} percent '+
        'of GDP (see {\color{green!85!black}\\textbf{---}}), compared '+
        f'to {fndpr:.1f} percent of GDP in {prdate}. Federal spending '+
        f'on national defense was equivalent to {dlt:.1f} percent of '+
        f'GDP in the latest quarter and {dpr:.1f} percent in {prdate} '+
        '(see {\color{blue!60!black}\\textbf{---}}). National defense '
        f'spending was {d89:.1f} percent of GDP in 1989 Q1. \n\n In '+
        f'{ltdate}, state and local government spending and investment '+
        f'was equivalent to {slt:.1f} percent of GDP, compared to '+
        f'{spr:.1f} percent in {prdate} '+
        '(see {\color{purple!70!magenta}\\textbf{---}}).')
write_txt(text_dir / 'govci.txt', text)
print(text)

In 2023 Q3, federal non-defense spending and investment was \$782.1 billion, equivalent to 2.8 percent of GDP (see {\color{green!85!black}\textbf{---}}), compared to 2.6 percent of GDP in 2019 Q4. Federal spending on national defense was equivalent to 3.6 percent of GDP in the latest quarter and 4.0 percent in 2019 Q4 (see {\color{blue!60!black}\textbf{---}}). National defense spending was 6.9 percent of GDP in 1989 Q1. 

 In 2023 Q3, state and local government spending and investment was equivalent to 10.8 percent of GDP, compared to 11.1 percent in 2019 Q4 (see {\color{purple!70!magenta}\textbf{---}}).


### Consumer Spending Growth Contributions

In [61]:
s = ['B230RC']
population = nipa_df(retrieve_table('T20100')['Data'], s)['B230RC']
s = ['DPCERG']
d = nipa_df(retrieve_table('T20304')['Data'], s)['DPCERG']
deflator = d.iloc[-1] / d

s = ['A067RC', 'A068RC', 'A071RC', 'DPCERC']

df = (nipa_df(retrieve_table('T20100')['Data'], s)
      .assign(OTHER = lambda x: -(x['A068RC'] - x['DPCERC']),
              SAVING = lambda x: -x['A071RC'])
      .drop(['A068RC'], axis=1)
      .divide(population, axis=0)
      .multiply(deflator, axis=0))

data = growth_contrib(df, 'DPCERC').rolling(4).mean()
data3y = growth_contrib(df, 'DPCERC').rolling(12).mean()
data.loc['1989':].to_csv(data_dir / 'pcedecomp.csv', index_label='date')

In [62]:
date = f'{data.index[-1].year} Q{data.index[-1].quarter}'
pcetext = value_text(data['DPCERC'].iloc[-1], adj='average')

slist = ['A067RC', 'SAVING', 'OTHER']
d = {}
for i in slist:
    val = data[i].iloc[-1]
    vt = value_text(data[i].iloc[-1], style='contribution', 
           ptype='pp', casual=True)
    if f'{abs(val):.1f}' == '0.0':
        vt = "didn't affect the total"
    d[i] = vt
pce19 = value_text(data.loc['2019', 'DPCERC'].mean(), adj='average')
dpi19 = value_text(data.loc['2019', 'A067RC'].mean(), style='contribution', 
           ptype='pp')
save19 = value_text(data.loc['2019', 'SAVING'].mean(), style='contribution', 
           ptype='pp')
ltsv = data['SAVING'].iloc[-1].round(2)
svid = 'decreased' if ltsv > 0 else '' if ltsv == 0 else 'increased'
ltot = data['OTHER'].iloc[-1].round(2)
otid = 'decreases in' if ltsv > 0 else '' if ltsv == 0 else 'increases in'
pcetxt = (f'Real per capita consumer spending {pcetext} over the '+
          f'four quarters ending {date}. Changes to disposable income '+
          f'{d["A067RC"]}, {svid} saving {d["SAVING"]}, and '+
          f'{otid} other outlays {d["OTHER"]}. '+
          f'During 2019, real per capita consumer spending {pce19}. '+
          f'Increased income {dpi19}, and a slight increase in saving {save19}.')
write_txt(text_dir / 'pcedecomp.txt', pcetxt)
print(pcetxt)

Real per capita consumer spending increased at an average rate of 1.9 percent over the four quarters ending 2023 Q3. Changes to disposable income added 3.5 percentage points, increased saving subtracted 0.8 percentage point, and increases in other outlays subtracted 0.8 percentage point. During 2019, real per capita consumer spending increased at an average rate of 1.5 percent. Increased income contributed 2.8 percentage points, and a slight increase in saving subtracted 1.2 percentage points.


### Sectoral Accounts

**NOTE:** Need to convert "deficit", "borrower" etc to parameters!

In [63]:
def sect_txt(val):
    lb = ('borrower', 'deficit') if val < 0 else ('lender', 'surplus')
    d = {'v': val,
         'vt': f'{val:.1f}',
         'avt': f'{abs(val):.1f}',
         'vg': f'the equivalent of {abs(val):.1f} percent of GDP',
         'lb': lb[0],
         'ds': lb[1]}
    return d

In [64]:
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
s = ['W162RC', 'W994RC', 'AD01RC', 'W995RC', 'W996RC', 'AD03RC']
sb = nipa_df(retrieve_table('T50100')['Data'], s)
df = (sb.div(gdp, axis=0) * 100).dropna()
res = df[['W995RC', 'W996RC', 'AD03RC']].loc['1989':]
res.to_csv(data_dir / 'sectbal2.csv', index_label='date')

data = pd.DataFrame()
data['PRIV'] = df['W994RC']
data['GOV'] = df['AD01RC']
data['ROW'] = -df['W162RC'] # Negative in raw data
data = data.dropna().loc['1989':]
data.to_csv(data_dir / 'sectbal.csv', index_label='date')

date = dtxt(data.index[-1])['qtr1']
pcdt = '2019-10-01'

priv = sect_txt(data.PRIV.iloc[-1])
privpr = sect_txt(data.loc['2019', 'PRIV'].mean())
row = sect_txt(data.ROW.iloc[-1])
rowpr = sect_txt(data.loc['2019', 'ROW'].mean())
gov = sect_txt(data.GOV.iloc[-1])
govpr = sect_txt(data.loc['2019', 'GOV'].mean())

compare = compare_text(priv['v'], privpr['v'], [0.4, 1.0, 3.5])

txt = (f"In {date}, the US private sector was a net {priv['lb']} "+
       f"(running a {priv['ds']}) of {priv['vg']}, {compare} the "+
       f"{privpr['avt']} percent surplus in 2019. The rest of the world "+
       f"was a net {row['lb']} to the US to {row['vg']} in {date}, compared "+
       f"to {rowpr['avt']} percent in 2019. Balancing these transactions, "+
       "the government (federal, state, and local combined) was a net "+
       f"{gov['lb']} (running a {gov['ds']}) of {gov['vg']} in {date}, "+
       f"compared to {govpr['avt']} percent in 2019. ")
write_txt(text_dir / 'sectbal.txt', txt)
print(txt, '\n')

date = dtxt(df.index[-1])['qtr1']
hh = sect_txt(df.W996RC.iloc[-1])
hhpr = sect_txt(df.loc['2019', 'W996RC'].mean())
pb = sect_txt(df.W995RC.iloc[-1])
pbpr = sect_txt(df.loc['2019', 'W995RC'].mean())   
text = ('Breaking out the two main categories in the private sector, households '+
       f'were net {hh["lb"]}s (ran a {hh["ds"]}) of {hh["vg"]} in {date} '+
        '(see\cbox{orange!90!yellow}), while private businesses--corporate '+
       f'and noncorporate--were net {pb["lb"]}s of {pb["vg"]} '+
       '(see\cbox{purple!50!red}). In 2019, households were net '+
       f'{hhpr["lb"]}s of {hhpr["avt"]} percent, and private businesses were net '+
       f'{pbpr["lb"]}s of {pbpr["avt"]} percent.') 
write_txt(text_dir / 'sectbal2.txt', text)
print(text)

In 2023 Q2, the US private sector was a net lender (running a surplus) of the equivalent of 4.6 percent of GDP, in line with the 4.6 percent surplus in 2019. The rest of the world was a net lender to the US to the equivalent of 3.1 percent of GDP in 2023 Q2, compared to 2.1 percent in 2019. Balancing these transactions, the government (federal, state, and local combined) was a net borrower (running a deficit) of the equivalent of 7.7 percent of GDP in 2023 Q2, compared to 6.7 percent in 2019.  

Breaking out the two main categories in the private sector, households were net lenders (ran a surplus) of the equivalent of 2.7 percent of GDP in 2023 Q2 (see\cbox{orange!90!yellow}), while private businesses--corporate and noncorporate--were net lenders of the equivalent of 1.9 percent of GDP (see\cbox{purple!50!red}). In 2019, households were net lenders of 4.0 percent, and private businesses were net lenders of 0.6 percent.


### Current Account Balance

In [65]:
s = ['A191RC']

gdp = nipa_df(retrieve_table('T10105')['Data'], s)

n = {'A124RC': 'Current Account Balance',
     'A1073C': 'Current Receipts',
     'B020RC': '\hspace{1mm}Exports',
     'A253RC': '\hspace{3mm}Goods',
     'A332RC': '\hspace{5mm}Durable',
     'A339RC': '\hspace{5mm}Non-Durable',
     'A646RC': '\hspace{3mm}Services',
     'B645RC': '\hspace{1mm}Income Receipts',
     'LA000035': '\hspace{1mm}Transfer Receipts',
     'W163RC': 'Current payments',
     'B021RC': '\hspace{1mm}Imports',
     'A255RC': '\hspace{3mm}Goods',
     'A333RC': '\hspace{5mm}Durable',
     'A340RC': '\hspace{5mm}Non-Durable',
     'B656RC': '\hspace{3mm}Services',
     'A655RC': '\hspace{1mm}Income Payments',
     'A123RC': '\hspace{1mm}Transfer Payments'}

s = ['A124RC', 'GOODS', 'SERVICES', 'INCOME', 'TRANSFERS']

df = (nipa_df(retrieve_table('T40100')['Data'], n.keys())
      .assign(GOODS = lambda x: x['A253RC'] - x['A255RC'],
              SERVICES = lambda x: x['A646RC'] - x['B656RC'],
              INCOME = lambda x: x['B645RC'] - x['A655RC'],
              TRANSFERS = lambda x: - x['A123RC']))

data = (df.div(nipa_df(retrieve_table('T10105')['Data'], ['A191RC']
               )['A191RC'], axis=0).loc['1989':].multiply(100).round(2))

data.loc['1989':, s].dropna().to_csv(data_dir / 'cab.csv', index_label='date')

node = end_node(data['A124RC'].dropna(), 'black', date='q')
write_txt(text_dir / 'cab_node.txt', node)  

cab = abs(data.dropna()['A124RC'].iloc[-1])
tb = abs(data.dropna()['GOODS'].iloc[-1])
ld = dtxt(data.dropna().index[-1])['qtr1']
ltdate = dtxt(data.index[-1])['qtr1']
prdate = dtxt(data.index[-2])['qtr1']
lttbval = abs(data['GOODS'].iloc[-1])
tbprval = abs(data['GOODS'].iloc[-2])
prval = abs(data['A124RC'].iloc[-2])

if pd.isna(data['A124RC'].iloc[-1]) == True:
    mtxt = (f'The initial GDP report for {ltdate} does not include the '+
            'data needed to calculate the current account balance, however, '+
            f'the goods trade deficit for {ltdate} is equivalent to '+
            f'{lttbval:.1f} percent of GDP. ')
else:
    mtxt = (f'In {prdate}, the current account deficit was equivalent to '
            f'{prval:.1f}, and the trade deficit was equivalent to '+
            f'{tbprval:.1f} percent of GDP.')

text = (f'As of {ld}, the US runs a current account deficit of {cab:.1f} '+
        'percent of GDP, primarily as the result of a trade deficit on '+
        f'goods of {tb:.1f} percent of GDP. {mtxt}')
write_txt(text_dir / 'cab.txt', text)
print(text)

# Table
result = data[n.keys()]
data2 = result.iloc[-6:].iloc[::-1].T

cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data2.columns)]

data2.columns = cols
data2['3-year'] = result.rolling(11).mean().iloc[-1].round(2)
data2['10-year'] = result.rolling(40).mean().iloc[-1].round(2)
data2.index = data2.index.map(n)
data2 = data2.applymap('{:.2f}'.format).replace('nan', '--')
data2.to_csv(data_dir / 'cab.tex', sep='&', lineterminator='\\\ ', quotechar=' ')

As of 2023 Q2, the US runs a current account deficit of 3.1 percent of GDP, primarily as the result of a trade deficit on goods of 4.0 percent of GDP. The initial GDP report for 2023 Q3 does not include the data needed to calculate the current account balance, however, the goods trade deficit for 2023 Q3 is equivalent to 3.9 percent of GDP. 


In [69]:
#nipa_series_codes(retrieve_table('T20100'))

### Corporate Profits Destination

In [66]:
s = ['A032RC', 'A438RC', 'A054RC', 'B056RC', 'A127RC']
df = (nipa_df(retrieve_table('T11200')['Data'], s)
         / 1_000_000).dropna().rename({})
df['NNI'] = df['A032RC'] - df['A438RC']
df['TOT'] = df[['A054RC', 'B056RC', 'A127RC']].sum(axis=1)
dt = df.index[-1]
ltdate = dtxt(df.index[-1])['qtr2']
sh = df.div(df.NNI, axis=0) * 100
(sh.loc['1989':, ['A054RC', 'B056RC', 'A127RC']]
   .to_csv(data_dir / 'cprof.csv', index_label='date'))
taxrt = ((df.A054RC / df.TOT) * 100).loc['1989':]
taxrt.to_csv(data_dir / 'cprof_taxrt.csv', index_label='date')
totsh19 = sh.loc['2019', 'TOT'].mean()
divsh19 = sh.loc['2019', 'B056RC'].mean()
resh19 = sh.loc['2019', 'A127RC'].mean()
taxsh19 = sh.loc['2019', 'A054RC'].mean()
taxrt19 = taxrt.loc['2019'].mean()
text = (f'In {ltdate}, corporate profits were '+
        f'\${df.TOT.iloc[-1]:.2f} trillion, equivalent to '+
        f'{sh.TOT.iloc[-1]:.1f} percent of the income paid '+
        'to US nationals after depreciation costs (net national '+
        f'income). Of this, \${df.B056RC.iloc[-1]:.2f} trillion, '+
        f'equivalent to {sh.B056RC.iloc[-1]:.1f} percent of '+
        'net national income, were paid out as dividends '+
        '(see\cbox{blue!70!purple}), '+
        f'\${df.A127RC.iloc[-1] * 1000:,.0f} billion were '+
        'retained (corporate saving, see\cbox{cyan!50!white}), '+
        f'and \${df.A054RC.iloc[-1] * 1000:.0f} billion, '+
        f'{taxrt.iloc[-1]:.1f} percent of corporate profits, '+
        'went to corporate income tax (see\cbox{red!80!orange}). \n\n'+
        f'In 2019, corporate profits were {totsh19:.1f} percent '+
        'of net national income. Dividends were equivalent to '+
        f'{divsh19:.1f} percent, corporate savings were '+
        f'{resh19:.1f} percent, and corporate income taxes were '+
        f'{taxsh19:.1f} percent of net national income '+
        f'and {taxrt19:.1f} percent of corporate profits.')
write_txt(text_dir / 'cprof.txt', text)
print(text)

cols = ['B056RC', 'A127RC', 'A054RC']
sdf = sh[cols].iloc[-1]
height = ((sdf.cumsum() - (sdf / 2) + 1.0)).to_dict()
val = sdf.to_dict()
dtp = dtxt(sh.index[-1] + pd.DateOffset(months=3))['datetime']
nodes = [f'\\absnode{{{{{dtp}}}}}{{{height[i]}}}{{\scriptsize {val[i]:.1f}}}' 
         for i in cols]
dtv = dtxt(sh.index[-1])['qtr4'].replace(' ', '\\\\ \scriptsize ')
dtn = f'\\absnode{{{{{dtp}}}}}{{{sdf.cumsum().iloc[-1] + 3.0}}}{{\scriptsize{dtv}:}}'
nodes.append(dtn)
nodetext = '\n'.join(nodes)
write_txt(text_dir / 'cprof_nodes.txt', nodetext)

In the second quarter of 2023, corporate profits were \$3.17 trillion, equivalent to 16.0 percent of the income paid to US nationals after depreciation costs (net national income). Of this, \$1.86 trillion, equivalent to 9.4 percent of net national income, were paid out as dividends (see\cbox{blue!70!purple}), \$746 billion were retained (corporate saving, see\cbox{cyan!50!white}), and \$570 billion, 18.0 percent of corporate profits, went to corporate income tax (see\cbox{red!80!orange}). 

In 2019, corporate profits were 15.1 percent of net national income. Dividends were equivalent to 8.6 percent, corporate savings were 4.6 percent, and corporate income taxes were 1.8 percent of net national income and 12.0 percent of corporate profits.


### Corporate profits source

In [67]:
# Based loosely on Levy's Where do Profits Come From?
s = ['W170RC', 'A262RC', 'W986RC', 'A922RC']
df1 = nipa_df(retrieve_table('T50100')['Data'], s)

s = ['A123RC']
df2 = nipa_df(retrieve_table('T40100')['Data'], s)

s = ['A001RC']
df3 = nipa_df(retrieve_table('T10705')['Data'], s)

cprof = pd.DataFrame()
cprof['ROW Saving'] = (df2['A123RC'] / df3['A001RC']) * 100
cprof['HH Saving'] = (- df1['W986RC'] / df3['A001RC']) * 100
cprof['Gov Saving'] = (- df1['A922RC'] / df3['A001RC']) * 100
cprof['Investment'] = ((df1['W170RC'] - df1['A262RC']) / df3['A001RC']) * 100

cprof.loc['1989':].to_csv(data_dir / 'cprof2.csv', index_label='date')

In [72]:
#nipa_series_codes(retrieve_table('T50100'))

### Labor Productivity ST / LT

In [68]:
# Sentence for comparison with median wage growth
df = fred_df('OPHNFB', start='1968')['VALUE']
lpch = ((df.iloc[-1] / df.loc['1989-01-01']) - 1) * 100
lpcht = value_text(lpch, 'increase_by')
text = ('Over this same period, which features a sharp increase '+
        f'in education in the US, labor productivity {lpcht}.')
write_txt(text_dir / 'lprod_rw_educ.txt', text)
print(text)

# Long-term and short-term growth rates
df = growth_rate(df)
lt = df.rolling(80).mean().dropna()
st = df.rolling(4).mean().dropna()
data = pd.DataFrame({'ST': st, 'LT': lt})
data.loc['1989':].to_csv(data_dir / 'prod_st_lt.csv', index_label='date')
ltdt = dtxt(data.index[-1])['qtr1']
ltavg = value_text(data['LT'].mean(), 'plain')
tty = value_text(data['LT'].iloc[-1], 'plain')
ltv = value_text(data['ST'].iloc[-1], 'plain')
colors = {'LT': 'orange!80!yellow', 'ST': 'blue!80!black'}
cl = {name: c_line(col) for name, col in colors.items()}
text = ('Over the longer-term, US labor productivity growth '+
        f'averages {ltavg} per year. The trailing 20-'+
        f'year average growth rate is {tty} in {ltdt} '+
        f'{cl["LT"]}. During the 1990s and early 2000s, labor '+
        'productivity growth was above its long-term average. '+
        'In contrast, from 2010 to 2017, productivity growth '+
        f'was below average. Over the year ending {ltdt}, '+
        f'growth averages {ltv} {cl["ST"]}. ')
write_txt(text_dir / 'prod_st_lt.txt', text)
print(text)

Over this same period, which features a sharp increase in education in the US, labor productivity increased by 89.8 percent.
Over the longer-term, US labor productivity growth averages two percent per year. The trailing 20-year average growth rate is 1.6 percent in 2023 Q2 (see {\color{orange!80!yellow}\textbf{---}}). During the 1990s and early 2000s, labor productivity growth was above its long-term average. In contrast, from 2010 to 2017, productivity growth was below average. Over the year ending 2023 Q2, growth averages 1.3 percent (see {\color{blue!80!black}\textbf{---}}). 


### Labor Productivity

In [4]:
# Series stored as a dictionary
series = {'PRS85006093': 'value_index',
          'PRS85006092': 'value',
          'PRS85006032': 'hours',
          'PRS85006042': 'output',
          'PRS85006033': 'hours_index',
          'PRS84006113': 'business_ulc',
          'PRS30006113': 'manuf_ulc',
          'PRS85006113': 'nfbus_ulc'}

In [5]:
# Start year and end year
dates = (1989, 2023)
df = bls_api(series, dates, bls_key)

df.to_csv(data_dir / 'lprod.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [70]:
df = pd.read_csv(data_dir / 'lprod.csv', 
                 index_col='date', parse_dates=True)
ltdt = dtxt(df.index[-1])['qtr1']
prdt = dtxt(df.index[-2])['qtr1']
s = {srs: {'lt': value_text(df[srs].iloc[-1], adj='annual'), 
           'pr': value_text(df[srs].iloc[-2], adj='annual'), 
           'lt2': value_text(df[srs].iloc[-1], style='increase_of'), 
           'pr2': value_text(df[srs].iloc[-2]),
           'pr3': value_text(df[srs].iloc[-2], style='increase_of')} 
     for srs in series.values()}
grlt = cagr(df['value_index'])
grltt = value_text(grlt, 'plain')
gr5 = cagr(df['value_index'].iloc[-20:])
gr5t = value_text(gr5, adj='annual')
compare = compare_text(gr5, grlt, [0.1, 0.5, 2.0])

text = (f'In {ltdt}, nonfarm business labor productivity {s["value"]["lt"]} '+
        f'(see\cbox{{teal}}), as the result of {s["output"]["lt2"]} in '+
        f'real output and {s["hours"]["lt2"]} in hours worked. In the '+
        f'prior quarter, {prdt}, labor productivity {s["value"]["pr"]}, '+
        f'as real output {s["output"]["pr2"]} and hours of work '+
        f'{s["hours"]["pr2"]}. Productivity has {gr5t} over the past '+
        f'five years, {compare} the 1989-onward rate of {grltt}.')
write_txt(text_dir / 'lprod.txt', text)
print(text, '\n')

hr19 = value_text(df.loc['2017': '2019', 'hours'].mean(), adj='average')
ch19 = (df['hours_index'].iloc[-1] / 
        df.loc['2019-10-01', 'hours_index'] - 1) * 100
ch19txt = value_text(ch19, 'increase_by', adj='total')
text = (f'Total hours worked in nonfarm businesses {s["hours"]["lt"]} '+
        f'in {ltdt}, following {s["hours"]["pr3"]} in {prdt}. From '+
        f'2017 through 2019, total hours worked {hr19}. Since 2019, '+
        f'hours worked have {ch19txt}.')
write_txt(text_dir / 'tot_hours.txt', text)
print(text)

In 2023 Q2, nonfarm business labor productivity increased at an annual rate of 3.5 percent (see\cbox{teal}), as the result of an increase of 1.9 percent in real output and a decrease of 1.5 percent in hours worked. In the prior quarter, 2023 Q1, labor productivity decreased at an annual rate of 1.2 percent, as real output increased 1.4 percent and hours of work increased 2.6 percent. Productivity has increased at an annual rate of 1.5 percent over the past five years, slightly below the 1989-onward rate of 1.9 percent. 

Total hours worked in nonfarm businesses decreased at an annual rate of 1.5 percent in 2023 Q2, following an increase of 2.6 percent in 2023 Q1. From 2017 through 2019, total hours worked increased at an average rate of 1.4 percent. Since 2019, hours worked have increased by a total of 2.3 percent.


### Gross Labor Income

In [71]:
df = (pd.read_csv(data_dir / 'jobs_report_main2.csv', parse_dates=[0])
         .set_index('date')[['avghrstot', 'EMPsa']])
emp = (df['avghrstot'] * df['EMPsa']).rename('Total')
coe = nipa_df(retrieve_table('T20100')['Data'], ['A033RC'])
data = coe.join(emp.resample('QS').mean()).dropna()
data['coe_inp'] = data['A033RC'] / data['Total']
data['wage'] = data['coe_inp'] * data['Total'].iloc[0]
data['work'] = data['A033RC'] - data['wage']

# Calculate contributions to growth
result = (growth_contrib(data, 'A033RC')[['work', 'wage']]
          .rolling(4).mean().dropna())
result['sum'] = result.sum(axis=1)
result.to_csv(data_dir / 'gli.csv', index_label='date')

# Horizontal bar at 5
start = dtxt(result.index[0] - pd.DateOffset(months=1))['datetime']
end = dtxt(result.index[-1] + pd.DateOffset(months=3))['datetime']
hbar = (f'\draw [dotted, thick] (axis cs:{{{start}}}, 5) -- '+
        f'(axis cs:{{{end}}}, 5);')
write_txt(text_dir / 'gli_hbar2.txt', hbar) 

# Text
ltdate = dtxt(result.index[-1])['qtr1']
totch = value_text(result['sum'].iloc[-1], adj='avg_ann', 
                   threshold=0.1)
wage = result['wage'].iloc[-1]
work = result['work'].iloc[-1]
    
txt2 = value_text(wage, 'contribution', 'pp')
txt3 = value_text(work, 'contribution', 'pp')

text = (f'{totch} over the year ending {ltdate}. Changes in wages {txt2}, '+
        f'and changes in total hours worked {txt3}.')
write_txt(text_dir / 'gli.txt', text)
print(text)

increased at an average annualized rate of 5.6 percent over the year ending 2023 Q3. Changes in wages contributed three percentage points, and changes in total hours worked contributed 2.5 percentage points.


### Financial Obligations Ratio

In [37]:
# Retrieve data and store
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=FOR&series=1dc13603606b1a2cf3c07004eeb7f026&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2023&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, clean_data = clean_fed_data(url)
d = {k: v.replace(', seasonally adjusted', '') for k, v in d.items()}
data = clean_data.rename(d, axis=1)
data.to_csv(data_dir / 'for.csv', index_label='date')

In [38]:
# End nodes and text
data = pd.read_csv(data_dir / 'for.csv', index_col='date', 
                   parse_dates=True)
# End nodes
cols = {'Financial obligations ratio': ['blue!80!black', 'q', 0.35], 
        'Debt service ratio': ['red', None, 0]}
nodes = '\n'.join([end_node(data[n], col, date=date, offset=offset) 
                   for n, [col, date, offset] in cols.items()])
write_txt(text_dir / 'for_nodes.txt', nodes)

# Text
fo = data['Financial obligations ratio']
ds = data['Debt service ratio']
ltdt = dtxt(data.index[-1])['qtr1']
ltval = fo.iloc[-1]
pryrval = fo.iloc[-5]
chval = value_text(ltval - pryrval, 'increase_of', ptype='pp', 
                   digits=2, threshold=0.03)
ch19 = value_text(ltval - fo.loc['2019-10-01'], 'increase_by', ptype='pp', 
                  digits=2, casual=True, threshold=0.03)
dsrval = ds.iloc[-1]
dch19 = value_text(dsrval - ds.loc['2019-10-01'], 'increase_by', ptype='pp', 
                   digits=2, casual=True, threshold=0.03).replace('was', 'is')
val90 = value_text(fo.loc['1990': '1999'].mean(), 'plain')
maxval = value_text(fo.max(), 'plain')
maxdt = dtxt(fo.idxmax())['qtr1']
hb = ', during the housing bubble' if fo.idxmax().year == 2007 else ''
m = data['Mortgage debt service ratio']
c = data['Consumer debt service ratio']
mltval = value_text(m.iloc[-1], 'plain')
cltval = value_text(c.iloc[-1], 'plain')
cl1 = c_line(cols['Financial obligations ratio'][0])
cl2 = c_line(cols['Debt service ratio'][0])

text = ('In the 1990s, financial obligations comprise an average of '+
        f'{val90} of income. This ratio peaked at {maxval} '+
        f'in {maxdt}{hb}.\n\n'+
        f'As of {ltdt}, household \\textbf{{financial obligations}} '+
        f'are {ltval:.1f} percent of income {cl1}, '+
        f'{chval} from the year prior. The financial obligations ratio '+
        f'{ch19} since 2019.\n\nIn the latest quarter, the ratio '+
        f'of \\textbf{{debt service}} payments to income is '+
        f'{dsrval:.1f} percent {cl2}. The debt service ratio '+
        f'{dch19} since 2019. In {ltdt}, the ratio of mortage debt '+
        f'service to income is {mltval}, and the ratio of '+
        f'consumer credit debt service to income is {cltval}.')
write_txt(text_dir / 'for.txt', text)
print(text)

In the 1990s, financial obligations comprise an average of 16.4 percent of income. This ratio peaked at 18.0 percent in 2007 Q4, during the housing bubble.

As of 2023 Q2, household \textbf{financial obligations} are 14.5 percent of income (see {\color{blue!80!black}\textbf{---}}), a decrease of 0.13 percentage point from the year prior. The financial obligations ratio fell by 0.23 percentage point since 2019.

In the latest quarter, the ratio of \textbf{debt service} payments to income is 9.8 percent (see {\color{red}\textbf{---}}). The debt service ratio is virtually unchanged since 2019. In 2023 Q2, the ratio of mortage debt service to income is four percent, and the ratio of consumer credit debt service to income is 5.8 percent.


### Shiller real return trailing 20-year average

In [15]:
url = 'http://www.econ.yale.edu/~shiller/data/ie_data.xls'
data = pd.read_excel(url, sheet_name='Data', header=7, 
                     index_col='Date').dropna(subset=['TR CAPE'])
data.index = pd.to_datetime(data.index.format())
data.loc['1989':, 'TR CAPE'].to_csv(data_dir / 'catrpe.csv', 
                                    index_label='date')
color = 'blue!80!cyan'
cl = c_line(color)
pe = data.loc['1989':, 'TR CAPE']
ltdt = dtxt(pe.index[-1])['mon1']
prdt = dtxt(pe.index[-2])['mon1']
yrdt = dtxt(pe.index[-13])['mon1']
ltval = pe.iloc[-1]
prval = pe.iloc[-2]
yrval = pe.iloc[-13]
val19 = pe.loc['2019'].mean()
val00 = pe.loc['2000'].mean()

text = (f'In {ltdt}, the Shiller total return CAPE ratio was '+
        f'{ltval:.1f}, compared to {prval:.1f} in {prdt} and '+
        f'{yrval:.1f} in {yrdt} {cl}. In 2019, the Shiller CAPE ratio '+
        f'was {val19:.1f}, on average. In 2000, during the stock market '+
        f'bubble, the ratio averaged {val00:.1f}. ')
write_txt(text_dir / 'cape.txt', text)
print(text)
node = end_node(data['TR CAPE'], color, date='m')
write_txt(text_dir / 'cape_node.txt', node)
col = ['Price', 'Dividend']
df = data.loc['1960':, col].dropna()
for yrs in [10, 15, 20]:
    mos = yrs * 12
    dy = (df.Dividend / df.Price).rolling(mos).mean()
    pch = (df.Price.pct_change(mos)+1)**(1/yrs) - 1
    df[f'r{yrs}'] = (dy + pch) * 100
    
res = df.loc['1989':, ~df.columns.isin(col)]
res.to_csv(data_dir / f'sp500rr.csv', index_label='date', 
           float_format='%g')
adj = node_adj(res)
smax = res.iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

colors = {'r20': 'green!80!blue', 
          'r15': 'orange',
          'r10': 'blue'}
date = {series: 'm' if series == smax else None 
        for series in colors.keys()}
nodes  ='\n'.join([end_node(res[series], color, 
                            date=date[series], 
                            size=1.1, offset=adj[series]) 
                   for series, color in colors.items()])
write_txt(text_dir / 'sp500rr_nodes.txt', nodes)  
url = 'http://www.econ.yale.edu/~shiller/data.htm'
lt20 = df['r20'].iloc[-1]
ltdt = dtxt(df.index[-1])['mon1']
pr = df['r20'].loc['1995':'2005'].mean()
lt10 = df['r10'].iloc[-1]
pr10 = df['r10'].loc['1995':'2005'].mean()
cl = c_line(colors['r20'])
text = (f'According to historical stock market \href{{{url}}}'+
        '{data} from Robert Shiller, the \\textbf{inflation-adjusted '+
        'trailing twenty-year annual rate of return} of the S\&P '+
        f'500 is {lt20:.1f} percent as of {ltdt} {cl}. Ultra-long-'+
        'term real returns are currently low relative to the '+
        'average trailing twenty-year real annual return of '+
        f'{pr:.1f} percent during 1995--2005. The trailing ten-'+
        f'year real return was {lt10:.1f} percent, as of {ltdt}, '+
        f'and {pr10:.1f} percent during 1995--2005 '+
        f'{c_line(colors["r10"])}. ')
write_txt(text_dir / 'sp500rr.txt', text)
print(text)

In September 2023, the Shiller total return CAPE ratio was 33.3, compared to 32.9 in August 2023 and 30.6 in September 2022 (see {\color{blue!80!cyan}\textbf{---}}). In 2019, the Shiller CAPE ratio was 32.1, on average. In 2000, during the stock market bubble, the ratio averaged 45.1. 
Three conflicting nodes
According to historical stock market \href{http://www.econ.yale.edu/~shiller/data.htm}{data} from Robert Shiller, the \textbf{inflation-adjusted trailing twenty-year annual rate of return} of the S\&P 500 is 6.9 percent as of June 2023 (see {\color{green!80!blue}\textbf{---}}). Ultra-long-term real returns are currently low relative to the average trailing twenty-year real annual return of 10.1 percent during 1995--2005. The trailing ten-year real return was 9.3 percent, as of June 2023, and 10.7 percent during 1995--2005 (see {\color{blue}\textbf{---}}). 


In [16]:
dy = ((data.D / data.P).dropna() * 100).loc['1989':].rename('DY')
dy.to_csv(data_dir / 'sp500dy.csv', index_label='date')
color = 'green!80!black'

node = end_node(dy, color, date='m', digits=2, offset=0.3)
write_txt(text_dir / 'sp500dy_node.txt', node)

ltdt = dtxt(dy.index[-1])['mon1']
prdt = dtxt(dy.index[-2])['mon1']
yrdt = dtxt(dy.index[-13])['mon1']
ltval = dy.iloc[-1]
prval = dy.iloc[-2]
yrval = dy.iloc[-13]
cl = c_line(color)
avg = dy.loc['1990': '2015'].mean()
text = (f'In {ltdt}, the dividend yield for the S\&P 500 is '+
        f'{ltval:.2f} percent {cl}, compared to {prval:.2f} percent '+
        f'in {prdt}, and {yrval:.2f} percent in {yrdt}. From 1990 to '+
        f'2015, the dividend yield averaged {avg:.2f} percent.')
write_txt(text_dir / 'sp500div.txt', text)
print(text)

In June 2023, the dividend yield for the S\&P 500 is 1.58 percent (see {\color{green!80!black}\textbf{---}}), compared to 1.65 percent in May 2023, and 1.64 percent in June 2022. From 1990 to 2015, the dividend yield averaged 2.09 percent.


### High Quality Corporate Bond Yield

In [82]:
df = fred_df('HQMCB10YR')

# Spread between Corporate Bonds and Treasuries 
dft = pd.read_csv(data_dir / 'treas_raw.csv', index_col='date', 
                 parse_dates=True).loc['1989':, 'Ten-year']

df['Treasury'] = dft.resample('MS').mean()
df['Spread'] = df['VALUE'] - df['Treasury']
df[['VALUE', 'Spread']].to_csv(data_dir / 'hqcb.csv', 
                               index_label='date')

color = 'violet!80!black'
node = end_node(df['VALUE'], color, date='m', digits=2)
write_txt(text_dir / 'hqcb_node.txt', node)

data = df.VALUE.apply('{:.2f} percent'.format).values
idx = [dtxt(i)['mon1'] for i in df.index]
cl = c_line(color)

text = (f'The yield on high-quality corporate bonds with a maturity '+
        f'of 10 years is {data[-1]} in {idx[-1]}, following '+
        f'{data[-2]} in {idx[-2]} {cl}. One year prior, in '+
        f'{idx[-13]}, this spot rate was {data[-13]}, and '+
        f'four years prior, in {idx[-49]}, it was {data[-49]}.')
write_txt(text_dir / 'hqcb.txt', text)
print(text)

The yield on high-quality corporate bonds with a maturity of 10 years is 5.65 percent in September 2023, following 5.50 percent in August 2023 (see {\color{violet!80!black}\textbf{---}}). One year prior, in September 2022, this spot rate was 5.27 percent, and four years prior, in September 2019, it was 2.93 percent.


### International Investment Position (IIP)

In [72]:
# Annual GDP for 1988-2005
s = 'A191RC'
r = bea_api_nipa([f'T10105'], bea_key, freq='A')
data = json.loads(r[0][2])['BEAAPI']['Results']
date = lambda x: (pd.to_datetime(x.TimePeriod) + 
                  pd.DateOffset(months=6))
value = lambda x: x.DataValue.str.replace(',','')
gdpa = (pd.DataFrame(data['Data']).query('SeriesCode == @s')
          .assign(date = date, value=value).set_index('date')
          .loc[:'2005', 'value'].astype(int))
gdpq = nipa_df(retrieve_table('T10105')['Data'], [s])[s]
gdp = pd.concat([gdpa, gdpq.loc['2006':]])

srs = ['FinAssets', 'FinLiabs', 'Net']
years = ','.join(map(str, range(1988, 2024)))
res = pd.DataFrame()
for s in srs:
    url = (f'https://apps.bea.gov/api/data/?&UserID={bea_key}'+
           f'&method=GetData&datasetname=IIP&TypeOfInvestment={s}'+
           f'&Component=Pos&Frequency=A,QNSA&Year={years}')
    r = requests.get(url)
    t = pd.DataFrame(r.json()['BEAAPI']['Data'])
    a = t.query('Frequency == "A"')[['TimePeriod', 'DataValue']]
    a = a.set_index(pd.to_datetime(a['TimePeriod']) + 
                    pd.DateOffset(months=6)).loc[:'2005', 'DataValue']
    q = t.query('Frequency == "QNSA"')[['TimePeriod', 'DataValue']]
    q = q.set_index(pd.to_datetime(q['TimePeriod']))['DataValue']
    res[s] = pd.concat([a, q]).astype('float')

sh = res.divide(gdp, axis=0).multiply(100).dropna()
sh.loc['1989':].to_csv(data_dir / 'iip.csv', index_label='date')

tot = res.iloc[-1] / 1_000_000
lt = sh.iloc[-1]
ltdt = dtxt(sh.index[-1])['qtr1']
pr = sh.iloc[-2]
prdt = dtxt(sh.index[-2])['qtr1']
pr19 = sh.loc['2019'].mean()
col_a, col_l, col_n = 'blue!95!violet', 'red', 'cyan!25!white'
cb = c_box(col_n).replace('see ', 'see')
text = (f'In {ltdt}, domestic holdings of foreign assets total '+
        f'\${tot.FinAssets:.1f} trillion, equivalent to {lt.FinAssets:.1f} '+
        f'percent of GDP {c_line(col_a)}. In {prdt}, these assets were '+
        f'equivalent to {pr.FinAssets:.1f} percent of GDP, and in '+
        f'2019, they were equivalent to {pr19.FinAssets:.1f} percent. '+
        'Domestic liabilities to the foreign sector total '+
        f'\${tot.FinLiabs:.1f} trillion, or {lt.FinLiabs:.1f} percent of '+
        f'GDP, in {ltdt}, following {pr.FinLiabs:.1f} percent in {prdt}, '+
        f'and {pr19.FinLiabs:.1f} percent in 2019 {c_line(col_l)}.\n\n'+
        f'The overall result of these financial positions, net IIP, or '+
        'holdings of foreign assets minus liabilities, identifies the '+
        'US as a net debtor to the rest of the world, to the equivalent '+
        f'of {abs(lt.Net):.1f} percent of GDP in {ltdt}, following '
        f'{abs(pr.Net):.1f} percent in {prdt}, and {abs(pr19.Net):.1f} '+
        f'percent in 2019 {cb}.')
write_txt(text_dir / 'niip.txt', text)
print(text)

In 2023 Q2, domestic holdings of foreign assets total \$33.6 trillion, equivalent to 124.1 percent of GDP (see {\color{blue!95!violet}\textbf{---}}). In 2023 Q1, these assets were equivalent to 122.0 percent of GDP, and in 2019, they were equivalent to 128.8 percent. Domestic liabilities to the foreign sector total \$51.6 trillion, or 190.6 percent of GDP, in 2023 Q2, following 184.4 percent in 2023 Q1, and 180.2 percent in 2019 (see {\color{red}\textbf{---}}).

The overall result of these financial positions, net IIP, or holdings of foreign assets minus liabilities, identifies the US as a net debtor to the rest of the world, to the equivalent of 66.5 percent of GDP in 2023 Q2, following 62.4 percent in 2023 Q1, and 51.4 percent in 2019 (see\cbox{cyan!25!white}).


### H.6 Money Stock - M2 (Monthly)

In [73]:
url = ('https://www.federalreserve.gov/datadownload/Output.aspx?'+
       'rel=H6&series=411c4c269dc600450339f8d4809d80eb&lastobs=&'+
       'from=01/01/1987&to=12/31/2023&filetype=csv&label=include&'+
       'layout=seriescolumn')
d, df = clean_fed_data(url)
df.rename(d, axis=1).to_csv(data_dir / 'h6raw.csv', index_label='date')

In [74]:
df = pd.read_csv(data_dir / 'h6raw.csv', index_col='date', 
                 parse_dates=True)
ltdate = dtxt(df.index[-1])['mon1']
prmodt = dtxt(df.index[-2])['mon1']
ltval = df['M2'].iloc[-1] / 1000.0
one_yr = value_text(df['M2'].pct_change(12).iloc[-1] * 100, threshold=0.1)
pr_mo = value_text(df['M2'].pct_change(12).iloc[-2] * 100, 
                   style='increase_of')
four_yr = value_text(df['M2'].pct_change(48).iloc[-1] * 100)

# M2 share of GDP
m2q = df.M2.resample('QS').mean() * 1_000
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
m2gdp = ((m2q / gdp) * 100).dropna()
m2gi = (m2gdp / m2gdp.iloc[0]).rename('value')
m2gi.to_csv(data_dir / 'm2gdp.csv', index_label='date',
         float_format='%g')
ltqtr = dtxt(m2gi.index[-1])['qtr2']
lt = m2gdp.iloc[-1]
totch = (m2gi.iloc[-1] - 1) * 100

txt1 = (f'In {ltdate}, the M2 money stock totals \${ltval:.1f} '+
        'trillion. Put into the context of overall economic activity, '+
        f'M2 is equivalent to {lt:.1f} percent of GDP in {ltqtr}.\n\nDuring '+
        'the 1990s, the ratio of money to economic activity was falling '+
        f'{c_line("magenta")}. Following the great recession, the money '+
        'supply has expanded relative to activity. Since 1989, the ratio '+
        f'has increased by a total of {totch:.1f} percent.')
write_txt(text_dir / 'm2lvl.txt', txt1)
col = 'green!80!blue'

mo_ch = value_text((df.M2.pct_change()*100).iloc[-1], threshold=0.1)
pc_mo_ch = value_text((df.M2.pct_change()*100).iloc[-2], 
                      'increase_of', threshold=0.1)
prdt = dtxt(df.index[-2])['mon1']
txt2 = (f'The M2 money stock {mo_ch} in {ltdate}, over the previous '+
        f'month, following {pc_mo_ch} in {prdt}. Over the past 12 '+
        f'months, the money stock {one_yr} {c_line(col)}. The M2 money '+
        f'stock has {four_yr}, in total, over the past four years. ')
write_txt(text_dir / 'm2chg.txt', txt2)
print(txt1, '\n', txt2)

r = pd.DataFrame({'value': df['M2'].pct_change(12) * 100,
                  '3M': m3rate(df['M2']),
                  '1M': (((df.M2.pct_change() + 1) ** 12) - 1) * 100}
                ).loc['1989':]
r.value.to_csv(data_dir / 'm2.csv', index_label='date', header=True,
         float_format='%g')

node = end_node(r.value, col, offset=0.35, date='m')
write_txt(text_dir / 'm2_node.txt', node)

In September 2023, the M2 money stock totals \$20.7 trillion. Put into the context of overall economic activity, M2 is equivalent to 75.0 percent of GDP in the third quarter of 2023.

During the 1990s, the ratio of money to economic activity was falling (see {\color{magenta}\textbf{---}}). Following the great recession, the money supply has expanded relative to activity. Since 1989, the ratio has increased by a total of 32.7 percent. 
 The M2 money stock decreased 0.1 percent in September 2023, over the previous month, following a decrease of 0.1 percent in August 2023. Over the past 12 months, the money stock decreased 3.6 percent (see {\color{green!80!blue}\textbf{---}}). The M2 money stock has increased 38.1 percent, in total, over the past four years. 


### Consumer Credit (G.19)

The same chart also includes a line from the Z.1 notebook

In [75]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=G19&series=8c7511a37e4aea678be81e7a61df57db&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2023&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, df = clean_fed_data(url)
cc = df['DTCTL.M']

dpi = (pd.read_csv(data_dir / 'nipa20600.csv', index_col='date', 
                   parse_dates=True).loc['1989':, 'A067RC'])
rate = ((cc / dpi) * 100).rename('value').dropna()
rate.to_csv(data_dir / 'cc_dpi_monthly.csv', index_label='date')

node_color = 'blue!50!black'
node = end_node(rate, node_color, date='m', offset=0.35)
write_txt(text_dir / 'cc_dpi_monthly_node.txt', node)

ltdate = dtxt(cc.index[-1])['mon1']
ltval = cc.iloc[-1] / 1_000_000
ltrate = rate.iloc[-1]
one_yr = value_text(cc.pct_change(12).iloc[-1] * 100, style='increase_by')
one_yr_dpi = value_text(dpi.pct_change(12).iloc[-1] * 100, style='increase_by')
one_yr_rate = value_text(rate.diff(12).iloc[-1], adj='total', 
                         ptype='pp', threshold=0.1)
cline = c_line(node_color)
also = 'also ' if one_yr == one_yr_dpi else ''    
text = ('In the monthly measure, consumer credit totals '+
        f'\${ltval:.2f} trillion US dollars on a '+
        f'seasonally-adjusted and annualized basis in {ltdate}. '+
        f'Over the past year, consumer credit {one_yr}, while '+
        f'after-tax income {also}{one_yr_dpi}. As a result, the ratio '+
        f'of consumer credit to disposable income {one_yr_rate}. '+
        f'In {ltdate}, total consumer credit is equivalent to '+
        f'{ltrate:.1f} percent of annualized {ltdate} disposable '+
        f'income {cline}. ')
write_txt(text_dir / 'cc_dpi.txt', text)
print(text)

In the monthly measure, consumer credit totals \$4.97 trillion US dollars on a seasonally-adjusted and annualized basis in August 2023. Over the past year, consumer credit increased by four percent, while after-tax income increased by 7.3 percent. As a result, the ratio of consumer credit to disposable income decreased by a total of 0.8 percentage point. In August 2023, total consumer credit is equivalent to 24.5 percent of annualized August 2023 disposable income (see {\color{blue!50!black}\textbf{---}}). 


### Employment Cost Index

In [6]:
series = {'CIU2020000000000A': 'WS',
          'CIU2030000000000A': 'Be',
          'CIS1020000000000I': 'IndexWS',
          'CIS102G000000000I': 'IndexWSGoods'}

dates = (2001, 2023)
df = bls_api(series, dates, bls_key)
df.to_csv(data_dir/ 'eci.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [7]:
df = pd.read_csv(data_dir / 'eci.csv', index_col='date', 
                 parse_dates=True)
adj = node_adj(df)
smax = df.iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

colors = {'WS': 'green!75!blue!60!black', 
          'Be': 'cyan!90!white'}
date = {series: 'q' if series == smax else None 
        for series in colors.keys()}
nodes  ='\n'.join([end_node(df[series], color, 
                            date=date[series], 
                            size=1.1, offset=adj[series])
                   for series, color in colors.items()])
write_txt(text_dir / 'eci_nodes.txt', nodes)

obs = [(-1, 'lt'), (-2, 'pr'), (-3, 'pr2')]
sty = [('increase', '1'), ('increase_of', '2'), 
       ('increase_by', '3')]
dt = {n: dtxt(df.index[i]) for i, n in obs}
v = {f'{name}_{n}_{s}': value_text(val, style=style) 
     for i, n in obs for name, val 
     in df.iloc[i].to_dict().items() 
     for style, s in sty}

val19 = {s: value_text(df.loc['2019', s].mean(), 'increase_by') 
         for s in ['WS', 'Be']}
text = (f'In {dt["lt"]["qtr2"]}, private industry wage and '+
        f'salary costs {v["WS_lt_3"]} (one-year percent '+
        f'change, {c_line(colors["WS"]).replace("(", "")}, '+
        f'following {v["WS_pr_2"]} in {dt["pr"]["qtr1"]}, '+
        f'and {v["WS_pr2_2"]} in {dt["pr2"]["qtr1"]}. In '+
        f'2019, private wages and salaries costs {val19["WS"]}, '+
        'on average.\n\nThe cost of private sector benefits '+
        f'{v["Be_lt_3"]} {c_line(colors["Be"])} over the year '+
        f'ending {dt["lt"]["qtr1"]}, following {v["Be_pr_2"]} '+
        f'in {dt["pr"]["qtr1"]}. In 2019, private-sector '+
        f'benefits costs {val19["Be"]}, on average.')
write_txt(text_dir / 'eci.txt', text)
print(text, '\n')

# Calculate Gross Labor Income
start = '2017-01-01'
eci = pd.read_csv(data_dir / 'eci.csv', index_col='date', 
                  parse_dates=True)['WS']
epop = (pd.read_csv(data_dir / 'jobs_report_main.csv', index_col='date', 
                   parse_dates=True)['PA_EPOP'].pct_change(12)
        .resample('QS').mean()) * 100
ece = (eci + epop).rename('ECI_EPOP').dropna()
# Shift dates to center quarter on chart
ds = ece.loc[start:]
ds.index = ds.index + pd.DateOffset(days=45)
ds.to_csv(data_dir / 'gli_qtr.csv', index_label='date')

#text
cline = c_line('violet!80!black')
ltdt = dtxt(ece.index[-1])['qtr1']
text = ('Calculating gross labor income from the employment cost index '+
        f'for private industries and the prime age employment rate {cline}, '+
        f'one-year growth is {ece.iloc[-1]:.1f} percent in {ltdt}, '+
        f'following {ece.iloc[-5]:.1f} percent one year prior. ')
write_txt(text_dir / 'gli_eci.txt', text)
print(text)

two sets of conflicting nodes
In the second quarter of 2023, private industry wage and salary costs increased by 4.6 percent (one-year percent change, see {\color{green!75!blue!60!black}\textbf{---}}), following an increase of 5.1 percent in 2023 Q1, and an increase of 5.1 percent in 2022 Q4. In 2019, private wages and salaries costs increased by three percent, on average.

The cost of private sector benefits increased by 3.9 percent (see {\color{cyan!90!white}\textbf{---}}) over the year ending 2023 Q2, following an increase of 4.3 percent in 2023 Q1. In 2019, private-sector benefits costs increased by two percent, on average. 

Calculating gross labor income from the employment cost index for private industries and the prime age employment rate (see {\color{violet!80!black}\textbf{---}}), one-year growth is 5.7 percent in 2023 Q2, following 9.3 percent one year prior. 


In [8]:
# Latest data for wage summary chart
df = pd.read_csv(data_dir / 'eci.csv', index_col='date', 
                 parse_dates=True)
cpi = pd.read_csv(data_dir / 'cpi_raw.csv', 
                 index_col='date', parse_dates=True)
cpidt = cpi.index[-73]
df.loc[cpidt:, 'WS'].to_csv(data_dir / 'eci_growth_recent.csv', 
                      index_label='date')

### Treasury International Capital

In [31]:
# URL 
file = ('https://treasury.gov/resource-center/data-chart-center/tic/'+
        'Documents/npr_history.csv')
#file = ('/home/brian/Downloads/npr_history.csv')
# Match names with numbers of columns in report
names = {10: 'Treasuries_official', 5: 'Treasuries_private', 
         11: 'Agencies_official', 6: 'Agencies_private', 
         12: 'Corporate_official', 7: 'Corporate_private',
         13: 'Equities_official', 8: 'Equities_private'}

df = pd.read_csv(file, header=12, index_col=0, parse_dates=True).iloc[1:].dropna()
df.index.name = 'date'
df.index = pd.to_datetime(df.index)
rn = {f'[{k}]': v for k, v in names.items()}
res = df[rn.keys()].rename(rn, axis=1).sort_index().loc['1988':]
# Categories to combine
c = {'Treasury_Bonds': ['Treasuries_official',
                        'Treasuries_private'],
     'Agency_Bonds': ['Agencies_official', 
                      'Agencies_private'],
     'Corporate_Bonds': ['Corporate_official',
                         'Corporate_private'],
     'Equities': ['Equities_official', 'Equities_private']}

final = pd.DataFrame()
for name, grp in c.items():
    final[name] = (res[grp].astype('int').sum(axis=1).sort_index())

In [32]:
s = ['Treasury_Bonds', 'Agency_Bonds', 'Corporate_Bonds']
pce = pd.read_csv(data_dir / 'nipa20804.csv', index_col='date', 
                  parse_dates=True).loc[final.index, 'DPCERG']
pr = pce / pce.iloc[-1]
data = (final[s].rolling(12).sum().loc['1989':]
        .divide(1000).dropna())
adj = (final[s].divide(pr, axis=0).rolling(12).sum().loc['1989':]
        .divide(1000).dropna())
adj.to_csv(data_dir/ 'tic_bond.csv', index_label='date')
date = dtxt(adj.index[-1])['mon1']
write_txt(text_dir / 'tic_date.txt', date)
print(date)

August 2023


In [33]:
ltdt = dtxt(data.index[-1])['mon1']
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
tbgdp = ((data.iloc[-1].loc['Treasury_Bonds'] * 1_000) / 
         gdp.iloc[-1]) * 100
tbgdpt = value_text(tbgdp, 'plain')

colors = {'Treasury_Bonds': 'black!25!white', 
          'Corporate_Bonds': 'blue!70!white',
          'Agency_Bonds': 'orange!30!yellow!90!white'}

cb = {k: c_box(v).replace('see ', 'see') for k,v in colors.items()}

d = {}
for i in s:
    item = i.replace('_', ' ').lower()
    val = data.iloc[-1].loc[i]
    buysell = 'buyer' if val > 0 else 'seller'
    txt = (f'a net {buysell} of \${val:,.0f} billion of US {item}' 
           if abs(val) >= 5 else 
           f'about unchanged in holdings of US {item}')
    d[i] = txt

text = (f'Over the year ending {ltdt}, the rest of the world was '+
        f'{d["Treasury_Bonds"]}, equivalent to {tbgdpt} '+
        f'of US GDP {cb["Treasury_Bonds"]}. Over the same period, '+
        f'the rest of the world was {d["Agency_Bonds"]}, '+
        f'{cb["Agency_Bonds"]}, and {d["Corporate_Bonds"]}, '+
        f'{cb["Corporate_Bonds"]}.')
write_txt(text_dir / 'tic.txt', text)
print(text)

Over the year ending August 2023, the rest of the world was a net buyer of \$582 billion of US treasury bonds, equivalent to 2.2 percent of US GDP (see\cbox{black!25!white}). Over the same period, the rest of the world was a net buyer of \$196 billion of US agency bonds, (see\cbox{orange!30!yellow!90!white}), and a net buyer of \$202 billion of US corporate bonds, (see\cbox{blue!70!white}).


### FRBNY Survey of Consumer Expectations Job Separation Expectation

In [34]:
url = ('https://www.newyorkfed.org/medialibrary/interactives/'+
       'sce/sce/downloads/data/frbny-sce-data.xlsx')
df = pd.read_excel(url, sheet_name='Job separation expectation', 
                   skiprows=3, index_col=0)
df.index = pd.to_datetime([f'{str(i)[:4]}-{str(i)[-2:]}-01' 
                          for i in df.index])
df.columns = ['Losing', 'Leaving']
df.to_csv(data_dir / 'sce_job_separation.csv', index_label='date')

In [35]:
df = pd.read_csv(data_dir / 'sce_job_separation.csv', index_col='date', 
                 parse_dates=True)

lcol = 'cyan'
vcol = 'violet!50!blue'
vnode = end_node(df.Leaving, vcol, date='m', offset=0.35)
lnode = end_node(df.Losing, lcol)
nodes = '\n'.join([vnode, lnode])
write_txt(text_dir / 'sce_job_sep_nodes.txt', nodes)

ltdt = dtxt(df.index[-1])['mon1']
ltv = df.Leaving.iloc[-1]
prv = df.Leaving.loc['2019'].mean()
ltl = df.Losing.iloc[-1]
prl = df.Losing.loc['2019'].mean()
diff = ltv - ltl
diffpr = prv - prl
exfb = 'exceed' if diff > 0 else 'false below'
text = (f'In {ltdt}, the perceived likelihood of leaving '+
        "one's job voluntarily in the next 12 months "+
        f'averages {ltv:.1f} percent, compared to {prv:.1f} '+
        f'percent in 2019 {c_line(vcol)}. In the latest month, '+
        f"the perceived probability losing one's job is {ltl:.1f} "+
        f'percent, compared to {prl:.1f} percent in 2019 '+
        f'{c_line(lcol)}.\n\nDuring the pandemic, in April 2020, '+
        'job loss expectations exceeded job leaving expectations. '+
        f'In {ltdt}, job leaving expectations {exfb} job loss '+
        f'expectations by {diff:.1f} percentage points, compared '+
        f'to {diffpr:.1f} percentage points in 2019. ')
write_txt(text_dir / 'sce_job_sep.txt', text)
print(text)

In September 2023, the perceived likelihood of leaving one's job voluntarily in the next 12 months averages 18.2 percent, compared to 21.0 percent in 2019 (see {\color{violet!50!blue}\textbf{---}}). In the latest month, the perceived probability losing one's job is 12.4 percent, compared to 14.3 percent in 2019 (see {\color{cyan}\textbf{---}}).

During the pandemic, in April 2020, job loss expectations exceeded job leaving expectations. In September 2023, job leaving expectations exceed job loss expectations by 5.8 percentage points, compared to 6.8 percentage points in 2019. 


### Real Effective Exchange Rate

From BIS

Index 2010 = 100

In [36]:
df = fred_df('RBUSBIS')
df.to_csv(data_dir / 'reer.csv', index_label='date')
ltdt = dtxt(df.index[-1])['mon1']
ltval = value_text(df.VALUE.iloc[-1] - 100)
val19 = df.loc['2019', 'VALUE'].mean()
lt3m = df.VALUE.iloc[-3:].mean()
url = 'https://www.bis.org/statistics/eer.htm'
text = (f'The Bank for International Settlements (BIS) \href{{{url}}}{{calculates}} '+
        '\\textbf{real effective exchange rates} for many countries, on a monthly '+
        f'basis. As of {ltdt}, the US dollar real effective exchange rate has '+
        f'{ltval} since 2010. In 2019, the index average was {val19:.1f}. Over the '+
        f'past three months, the index average value was {lt3m:.1f}.')
write_txt(text_dir / 'reer.txt', text)
print(text)
node = end_node(df.VALUE, 'red', date='m', full_year=True, offset=-0.35)
write_txt(text_dir / 'reer_node.txt', node)

The Bank for International Settlements (BIS) \href{https://www.bis.org/statistics/eer.htm}{calculates} \textbf{real effective exchange rates} for many countries, on a monthly basis. As of September 2023, the US dollar real effective exchange rate has increased 9.1 percent since 2010. In 2019, the index average was 98.6. Over the past three months, the index average value was 107.6.


### Table 1.15 Prices and Costs

In [95]:
d = nipa_series_codes(retrieve_table('T11500'))
df = nipa_df(retrieve_table('T11500')['Data'], d.keys())
res = growth_contrib_ann(df, 'A455RD')[['A460RD', 'A467RD', 'A463RD']]

In [96]:
res

,A460RD,A467RD,A463RD
1988-01-01,NaN,NaN,NaN
1988-04-01,NaN,NaN,NaN
1988-07-01,NaN,NaN,NaN
1988-10-01,NaN,NaN,NaN
1989-01-01,2.40,1.12,-0.32
...,...,...,...
2022-04-01,3.18,4.00,1.00
2022-07-01,3.06,3.24,1.80
2022-10-01,2.30,2.12,1.77
2023-01-01,2.33,0.95,0.43


### Construction Spending

Census data on the value of construction put-in-place

In [10]:
# Category codes and data type codes
d = {'PrRes': ['A00XX', 'V'], 'PrNR': ['ANRXX', 'V'], 
     'Pub': ['AXXXX', 'P'], 'Total': ['AXXXX', 'T'],
     'PrMan': ['A20IX', 'V']}
date = lambda x: pd.to_datetime(x.time)
df = pd.DataFrame()
for name, (cc, dtc) in d.items():
    url = ('https://api.census.gov/data/timeseries/eits/vip?'+
           f'get=cell_value,time_slot_id&key={census_key}&time=from+2002&'+
           f'category_code={cc}&data_type_code={dtc}&'+
           'for=us&seasonally_adj=yes')
    r = requests.get(url).json()
    df[name] = (pd.DataFrame(r[1:], columns=r[0]).assign(date = date)
                .set_index('date')['cell_value'].astype('float')).sort_index()
df.to_csv(data_dir / 'construction_spending.csv', index_label='date')

In [76]:
df = pd.read_csv(data_dir / 'construction_spending.csv', index_col='date', 
                 parse_dates=True)

gdp = pd.read_csv(data_dir / 'gdp_monthly.csv', index_col='date', 
                  parse_dates=True)['A191RC']

sh = (df.divide(gdp, axis=0).dropna() * 100)
sh.to_csv(data_dir / 'construction_spending_sh.csv', index_label='date')

# Text for total
ltdt = dtxt(sh.index[-1])['mon1']
ltval = f'\${(df.Total.iloc[-1] / 1_000_000):.1f} trillion'
ltsh = value_text(sh.Total.iloc[-1], 'eq', adj='GDP')

text = (f'In {ltdt}, the annualized value of construction put-in-place '+
        f'is {ltval}, {ltsh}.')
write_txt(text_dir / 'construction_spending_total.txt', text)
print(text)

# End nodes
colors = {'PrRes': 'blue!60!white', 'PrNR': 'yellow!60!orange', 
          'Pub': 'violet!60!black'}
res = sh[colors.keys()]
adj = node_adj(res)
smax = res.iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

date = {series: 'm' if series == smax else None 
        for series in colors.keys()}
nodes  ='\n'.join([end_node(res[series], color, 
                            date=date[series], full_year=True, 
                            size=1.1, offset=adj[series]) 
                   for series, color in colors.items()])
write_txt(text_dir / 'construction_spending_nodes.txt', nodes)  

# Share of GDP by sector
shd = {i: value_text(sh[i].iloc[-1], 'plain') 
       for i in ['PrRes', 'PrNR', 'Pub']}
cl = {i: c_line(c) for i, c in colors.items()}

# Growth contribution
df['GDP'] = gdp
gc = growth_contrib_ann(df, 'GDP', freq='M')
gcd = {i: value_text(gc[i].iloc[-1], 'contribution', 
                     ptype='pp', digits=2) 
       for i in ['PrRes', 'PrNR', 'Pub', 'Total']}
gcnr = value_text(gc['PrNR'].iloc[-1], 'contribution', 
                   ptype=None, digits=2) + ' point'
gcpub = value_text(gc['Pub'].iloc[-1], 'contribution', casual=True,
                   ptype=None, digits=2) + ' point'

text = (f'By sector, private residential construction is '+
        f'{shd["PrRes"]} of GDP {cl["PrRes"]} in {ltdt}, '+
        f'private nonresidential construction is {shd["PrNR"]} '+
        f'{cl["PrNR"]}, and government construction is '+
        f'{shd["Pub"]} {cl["Pub"]}.\n\n'+
        'Over the past year, construction spending '+
        f'{gcd["Total"]} to nominal GDP growth. Private residential '+
        f'construction {gcd["PrRes"]}, private '+
        f'nonresidential {gcnr}, and '+
        f'public construction {gcpub}.')
write_txt(text_dir / 'construction_spending_sector.txt', text)  
print(text)

In August 2023, the annualized value of construction put-in-place is \$2.0 trillion, equivalent to 7.1 percent of GDP.
By sector, private residential construction is 3.2 percent of GDP (see {\color{blue!60!white}\textbf{---}}) in August 2023, private nonresidential construction is 2.4 percent (see {\color{yellow!60!orange}\textbf{---}}), and government construction is 1.6 percent (see {\color{violet!60!black}\textbf{---}}).

Over the past year, construction spending contributed 0.53 percentage point to nominal GDP growth. Private residential construction subtracted 0.11 percentage point, private nonresidential contributed 0.43 point, and public construction added 0.21 point.
